> # Introduction <br>
Hi everyone! In this kernel I'd like to share some ideas about NN's and loss functions. <br>
Core points of this kernel: 
* Preparing tabular data for NN's
* Handling skewed continuous features
* Implementing custom loss function
* Which models are good for ensembling

# Data Loading and Feature Selection <br>
In this particular kernel we will use only features from transaction table, 'cause with NN after a brief investigation I didn't get any significant improvement by using all features.

In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='2'
!ls ../input/ieee-fraud-detection

sample_submission.csv  test_transaction.csv  train_transaction.csv
test_identity.csv      train_identity.csv


In [2]:
! ls ../input/ieee-fraud-detection/train_transaction.csv

../input/ieee-fraud-detection/train_transaction.csv


In [3]:
! ls ../input/ieee-pickled-dataset-from-hyunwookim

test_df.pkl  train_df.pkl


In [4]:
import pickle
train_df = pickle.load(open('./train_df_9551ka','rb'))
test_df = pickle.load(open('./test_df_9551ka','rb'))
y = train_df['isFraud']

In [5]:
import numpy as np
np.sum(y)

20663

In [6]:
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm_notebook
categorical_column_names = []
for col in tqdm_notebook(train_df.columns):
    if train_df[col].dtype == 'object' or col in ['addr1','addr2', 'email_type', 'm_comb','id_14']:
        categorical_column_names.append(col)
        le = LabelEncoder()
        le.fit(list(train_df[col].astype(str).values) + list(test_df[col].astype(str).values))
        train_df[col] = le.transform(list(train_df[col].astype(str).values))
        test_df[col] = le.transform(list(test_df[col].astype(str).values))   

In [7]:
len(train_df.columns)

567

In [8]:
categorical_column_names

['ProductCD',
 'card4',
 'card6',
 'addr1',
 'addr2',
 'P_emaildomain',
 'R_emaildomain',
 'M1',
 'M2',
 'M3',
 'M4',
 'M5',
 'M6',
 'M7',
 'M8',
 'M9',
 'email_type',
 'email_null_concat',
 'P_emaildomain_bin',
 'P_emaildomain_suffix',
 'R_emaildomain_bin',
 'R_emaildomain_suffix',
 'm_comb',
 'date',
 'card1_addr1',
 'card1_addr2',
 'card1_ProductCD',
 'TransactionAmt_ProductCD',
 'addr1_addr2',
 'id_12',
 'id_14',
 'id_15',
 'id_16',
 'id_23',
 'id_27',
 'id_28',
 'id_29',
 'id_30',
 'id_31',
 'id_33',
 'id_34',
 'id_35',
 'id_36',
 'id_37',
 'id_38',
 'DeviceType',
 'DeviceInfo',
 'same_Product_po']

In [9]:
features = [c for c in train_df.columns if c not in ['TransactionID', 'isFraud','TransactionDT','timeblock','account_start_day', 'date'
                                                    , 'year', 'month', 'target', 'day','account_make_date_D11', 'account_make_date_D3', 'account_make_date_D5', 'account_make_date_D4' , 'account_make_date_D8', 'account_make_date_D14', 'account_make_date_D6', 'account_make_date_D12', 'account_make_date_D7'
                                                    , 'card_1_2_3_5_nunique', 'card_1_2_3_5_prev_click', 'card_1_2_3_5_next_click', 'card_1_3_TransactionAmt_prev_click', 'card_1_3_TransactionAmt_next_click', 'account_make_date'
                                                    , 'poten_card1_nunique_D5', 'poten_card1_nunique_D11','poten_card1_nunique_D6', 'poten_card1_nunique_D3','poten_card1_nunique_D7','poten_card1_nunique_D12','poten_card1_nunique_D8','poten_card1_nunique_D4','poten_card1_nunique_D14'
                                                    , 'id_13', 'id_31', 'id_13_count_full', 'id_31_count_full','ProductCD', 'card3', 'card4', 'card5', 'card6', 'M1', 'M2', 'M3',
       'M4', 'M5', 'M7', 'M8', 'M9', 'P_emaildomain_bin',
       'P_emaildomain_suffix', 'R_emaildomain_bin',
       'R_emaildomain_suffix', 'account_make_date',
       'account_make_date_D3', 'account_make_date_D4',
       'account_make_date_D7', 'account_make_date_D8',
       'account_make_date_D11', 'account_make_date_D12',
       'account_make_date_D14', 'dayofweek', 'hour', 'card1_addr1',
       'card1_ProductCD', 'count_x', 'count_y', 'D15', "card1_TransactionAmt_mean",
        'card1_addr1hourstd','card1_addr1hourmedian','uid_hour_std','uid2_hour_std','card1_ProductCD_hour_std','card1_addr2_hour_std',
        'card1_TransactionAmt_nunique','card2_TransactionAmt_nunique','card3_TransactionAmt_nunique','card5_TransactionAmt_nunique','uid_TransactionAmt_nunique',
        'uid_hour_nunique','uid2_hour_nunique','card1_addr2_hour_nunique','card1_ProductCD_hour_nunique','account_make_date_D1','card1_year_month_mean','uid2_D4_mean','uid2_dayofweek_std','DT_M','Transactionhourcount']]

In [10]:
features

['TransactionAmt',
 'card1',
 'card2',
 'addr1',
 'addr2',
 'dist1',
 'dist2',
 'P_emaildomain',
 'R_emaildomain',
 'C1',
 'C2',
 'C3',
 'C4',
 'C5',
 'C6',
 'C7',
 'C8',
 'C9',
 'C10',
 'C11',
 'C12',
 'C13',
 'C14',
 'D1',
 'D2',
 'D3',
 'D4',
 'D5',
 'D6',
 'D7',
 'D8',
 'D9',
 'D10',
 'D11',
 'D12',
 'D13',
 'D14',
 'M6',
 'V1',
 'V2',
 'V3',
 'V4',
 'V5',
 'V6',
 'V7',
 'V8',
 'V9',
 'V10',
 'V11',
 'V12',
 'V13',
 'V14',
 'V15',
 'V16',
 'V17',
 'V18',
 'V19',
 'V20',
 'V21',
 'V22',
 'V23',
 'V24',
 'V25',
 'V26',
 'V27',
 'V28',
 'V29',
 'V30',
 'V31',
 'V32',
 'V33',
 'V34',
 'V35',
 'V36',
 'V37',
 'V38',
 'V39',
 'V40',
 'V41',
 'V42',
 'V43',
 'V44',
 'V45',
 'V46',
 'V47',
 'V48',
 'V49',
 'V50',
 'V51',
 'V52',
 'V53',
 'V54',
 'V55',
 'V56',
 'V57',
 'V58',
 'V59',
 'V60',
 'V61',
 'V62',
 'V63',
 'V64',
 'V65',
 'V66',
 'V67',
 'V68',
 'V69',
 'V70',
 'V71',
 'V72',
 'V73',
 'V74',
 'V75',
 'V76',
 'V77',
 'V78',
 'V79',
 'V80',
 'V81',
 'V82',
 'V83',
 'V84',
 'V85',
 

In [11]:
len(test_df.columns)

566

In [12]:
train_df = train_df[features]
train_df.head()

,TransactionAmt,card1,card2,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,...,D_mean,D_std,D_min,D_max,D_na_counts,same_Product_po,same_Product_po_cnt,same_Product_po_cnt_bydate,account_make_date_D1_count_full,TransactionPerHour
0,68.5,13926.0,NaN,91,48,19.0,NaN,32,32,1.0,...,NaN,NaN,NaN,NaN,5,51570,3,1,3856,0.006033
1,29.0,2755.0,404.0,96,48,NaN,NaN,16,32,1.0,...,NaN,NaN,NaN,NaN,5,54776,89,34,4297,0.006033
2,59.0,4663.0,490.0,98,48,287.0,NaN,36,32,1.0,...,NaN,NaN,NaN,NaN,5,54964,259,110,4297,0.006033
3,50.0,18132.0,567.0,153,48,NaN,NaN,55,32,2.0,...,NaN,NaN,NaN,NaN,5,37620,19,3,1621,0.006033
4,50.0,4497.0,514.0,128,48,NaN,NaN,16,32,1.0,...,NaN,NaN,NaN,NaN,5,54518,100,94,4297,0.006033


In [13]:
# describe_info = train_df.describe()

In [14]:
train_df.describe()

,TransactionAmt,card1,card2,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,...,D_mean,D_std,D_min,D_max,D_na_counts,same_Product_po,same_Product_po_cnt,same_Product_po_cnt_bydate,account_make_date_D1_count_full,TransactionPerHour
count,590540.000000,590540.000000,581607.000000,590540.000000,590540.000000,238269.0,37627.0,590540.000000,590540.000000,590540.0,...,118309.000000,75336.000000,118309.000000,118309.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000
mean,NaN,7967.812012,-501.946960,90.494251,48.558678,NaN,inf,24.851331,28.986617,NaN,...,0.000000,0.000000,-0.247981,0.422751,4.473077,73702.557071,90.817824,18.161286,5288.824279,0.006062
std,NaN,15246.676758,9277.760742,51.731095,2.462147,NaN,inf,16.850178,8.963983,NaN,...,0.000000,0.000000,0.764536,1.335665,1.247392,37793.622483,176.311954,55.472581,4017.081748,0.002178
min,0.250977,-99999.000000,-99999.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,...,-1.077148,0.000000,-2.013672,-1.075195,0.000000,0.000000,1.000000,1.000000,1.000000,0.000262
25%,43.312500,5812.000000,206.000000,43.000000,48.000000,3.0,7.0,16.000000,32.000000,1.0,...,-0.444824,0.109131,-0.578613,-0.324219,5.000000,46948.000000,4.000000,1.000000,1977.000000,0.004742
50%,68.750000,9561.000000,361.000000,82.000000,48.000000,8.0,37.0,16.000000,32.000000,1.0,...,-0.351318,0.163330,-0.461426,-0.235474,5.000000,73087.000000,24.000000,2.000000,4297.000000,0.005499
75%,125.000000,14080.000000,512.000000,134.000000,48.000000,24.0,206.0,32.000000,32.000000,3.0,...,0.182739,0.732422,-0.377441,0.788574,5.000000,103911.000000,100.000000,9.000000,7463.000000,0.006763
max,31936.000000,18396.000000,600.000000,184.000000,54.000000,10288.0,11624.0,60.000000,60.000000,4684.0,...,8.328125,6.191406,5.816406,12.062500,5.000000,212375.000000,1736.000000,606.000000,16700.000000,0.027468


In [15]:
train_df

,TransactionAmt,card1,card2,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,...,D_mean,D_std,D_min,D_max,D_na_counts,same_Product_po,same_Product_po_cnt,same_Product_po_cnt_bydate,account_make_date_D1_count_full,TransactionPerHour
0,68.500000,13926.0,NaN,91,48,19.0,NaN,32,32,1.0,...,NaN,NaN,NaN,NaN,5,51570,3,1,3856,0.006033
1,29.000000,2755.0,404.0,96,48,NaN,NaN,16,32,1.0,...,NaN,NaN,NaN,NaN,5,54776,89,34,4297,0.006033
2,59.000000,4663.0,490.0,98,48,287.0,NaN,36,32,1.0,...,NaN,NaN,NaN,NaN,5,54964,259,110,4297,0.006033
3,50.000000,18132.0,567.0,153,48,NaN,NaN,55,32,2.0,...,NaN,NaN,NaN,NaN,5,37620,19,3,1621,0.006033
4,50.000000,4497.0,514.0,128,48,NaN,NaN,16,32,1.0,...,NaN,NaN,NaN,NaN,5,54518,100,94,4297,0.006033
5,49.000000,5937.0,555.0,71,48,36.0,NaN,16,32,1.0,...,NaN,NaN,NaN,NaN,5,54917,137,52,4297,0.006033
6,159.000000,12308.0,360.0,10,48,0.0,NaN,55,32,1.0,...,NaN,NaN,NaN,NaN,5,54653,4,4,4297,0.006033
7,422.500000,12695.0,490.0,96,48,NaN,NaN,29,32,1.0,...,NaN,NaN,NaN,NaN,5,54879,7,5,4297,0.006033
8,15.000000,2803.0,100.0,101,48,NaN,NaN,1,32,1.0,...,NaN,NaN,NaN,NaN,5,54507,13,13,4297,0.006033
9,117.000000,17399.0,111.0,42,48,19.0,NaN,55,32,2.0,...,NaN,NaN,NaN,NaN,5,43866,71,2,880,0.006033


In [16]:
train_df.TransactionAmt.dtype

dtype('float16')

In [17]:
test_df = test_df[features]
test_df.head()

,TransactionAmt,card1,card2,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,...,D_mean,D_std,D_min,D_max,D_na_counts,same_Product_po,same_Product_po_cnt,same_Product_po_cnt_bydate,account_make_date_D1_count_full,TransactionPerHour
0,31.953125,10409.0,111.0,28,48,1.0,NaN,16,32,6.0,...,NaN,NaN,NaN,NaN,5,27087,3,1,1169,0.004704
1,49.000000,4272.0,111.0,82,48,4.0,NaN,2,32,3.0,...,NaN,NaN,NaN,NaN,5,88429,140,1,3710,0.004704
2,171.000000,4476.0,574.0,152,48,2636.0,NaN,19,32,2.0,...,NaN,NaN,NaN,NaN,5,93783,46,1,2979,0.004704
3,285.000000,10989.0,360.0,43,48,17.0,NaN,16,32,5.0,...,NaN,NaN,NaN,NaN,5,135468,3,1,4909,0.004704
4,67.937500,18018.0,452.0,67,48,6.0,NaN,16,32,6.0,...,NaN,NaN,NaN,NaN,5,140449,20,1,2351,0.004704


In [18]:
# train.describe()s

In [19]:
# import ctrNet
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
# from src import misc_utils as utils
import os
import gc
import random

In [20]:
np.unique([str(item) for item in test_df.dtypes.values])

array(['float16', 'float32', 'float64', 'int16', 'int32', 'int64', 'int8'],
      dtype='<U7')

In [21]:
# categorical features
# possible_categorical_features_train = np.array([
#     (item in ['ProductCD','card1','card2','card3','card4','card5','card6','addr1','addr2','P_emaildomain_bin'] for item in train_df.columns.values])
# train_cat = train_df.loc[:,possible_categorical_features_train]
# continuous features
# possible_continuous_features_train = np.array([('float64' in str(item)) for item in train_df.dtypes.values])

possible_categorical_features_train = np.array([item in categorical_column_names for item in train_df.columns.values])
possible_continuous_features_train = np.array([item not in categorical_column_names for item in train_df.columns.values])
train_cat = train_df.loc[:,possible_categorical_features_train]
train = train_df.loc[:,possible_continuous_features_train]
# remove outlier values
# rule_out_values = np.array([not ('std' in str(item)) for item in train_df.columns.values])
# possible_continuous_features_train = possible_continuous_features_train*rule_out_values
# train = train_df.loc[:,possible_continuous_features_train]

In [22]:
train.describe()

,TransactionAmt,card1,card2,dist1,dist2,C1,C2,C3,C4,C5,...,D_sum,D_mean,D_std,D_min,D_max,D_na_counts,same_Product_po_cnt,same_Product_po_cnt_bydate,account_make_date_D1_count_full,TransactionPerHour
count,590540.000000,590540.000000,581607.000000,238269.0,37627.0,590540.0,590540.0,590540.0,590540.0,590540.0,...,590540.000000,118309.000000,75336.000000,118309.000000,118309.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000
mean,NaN,7967.812012,-501.946960,NaN,inf,NaN,NaN,0.0,NaN,NaN,...,-0.000000,0.000000,0.000000,-0.247981,0.422751,4.473077,90.817824,18.161286,5288.824279,0.006062
std,NaN,15246.676758,9277.760742,NaN,inf,NaN,NaN,0.0,NaN,NaN,...,0.000000,0.000000,0.000000,0.764536,1.335665,1.247392,176.311954,55.472581,4017.081748,0.002178
min,0.250977,-99999.000000,-99999.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-2.361328,-1.077148,0.000000,-2.013672,-1.075195,0.000000,1.000000,1.000000,1.000000,0.000262
25%,43.312500,5812.000000,206.000000,3.0,7.0,1.0,1.0,0.0,0.0,0.0,...,0.000000,-0.444824,0.109131,-0.578613,-0.324219,5.000000,4.000000,1.000000,1977.000000,0.004742
50%,68.750000,9561.000000,361.000000,8.0,37.0,1.0,1.0,0.0,0.0,0.0,...,0.000000,-0.351318,0.163330,-0.461426,-0.235474,5.000000,24.000000,2.000000,4297.000000,0.005499
75%,125.000000,14080.000000,512.000000,24.0,206.0,3.0,3.0,0.0,0.0,1.0,...,0.000000,0.182739,0.732422,-0.377441,0.788574,5.000000,100.000000,9.000000,7463.000000,0.006763
max,31936.000000,18396.000000,600.000000,10288.0,11624.0,4684.0,5692.0,26.0,2252.0,349.0,...,25.812500,8.328125,6.191406,5.816406,12.062500,5.000000,1736.000000,606.000000,16700.000000,0.027468


In [23]:
train_cat.describe()

,addr1,addr2,P_emaildomain,R_emaildomain,M6,email_type,email_null_concat,m_comb,card1_addr2,TransactionAmt_ProductCD,...,id_30,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo,same_Product_po
count,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,...,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000
mean,90.494251,48.558678,24.851331,28.986617,0.900945,1.912587,1.087413,96.467049,5845.460133,5851.189936,...,69.816231,175.929453,2.837408,1.654289,1.534238,1.709557,1.636084,1.617342,1134.838951,73702.557071
std,51.731095,2.462147,16.850178,8.963983,0.814138,0.888649,0.888649,62.951641,3428.820896,3411.000688,...,12.331830,39.986861,0.446375,0.663430,0.838376,0.556321,0.694144,0.724336,178.855434,37793.622483
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,43.000000,48.000000,16.000000,32.000000,0.000000,2.000000,1.000000,30.000000,2864.000000,2576.000000,...,74.000000,190.000000,3.000000,2.000000,2.000000,2.000000,2.000000,2.000000,1187.000000,46948.000000
50%,82.000000,48.000000,16.000000,32.000000,1.000000,2.000000,1.000000,93.000000,5783.000000,6438.500000,...,74.000000,190.000000,3.000000,2.000000,2.000000,2.000000,2.000000,2.000000,1187.000000,73087.000000
75%,134.000000,48.000000,32.000000,32.000000,2.000000,2.000000,1.000000,139.000000,8920.000000,8877.000000,...,74.000000,190.000000,3.000000,2.000000,2.000000,2.000000,2.000000,2.000000,1187.000000,103911.000000
max,184.000000,54.000000,60.000000,60.000000,2.000000,3.000000,3.000000,255.000000,11560.000000,11580.000000,...,75.000000,190.000000,3.000000,2.000000,2.000000,2.000000,2.000000,2.000000,1214.000000,212375.000000


In [24]:
# C -> Inf
# np.array([('float16' in str(item)) for item in train_df.dtypes.values])*np.array([(str(item) in ['TransactionAmt','dist1', 'dist2','C1', 'C2', 'C3', 'C4', 'C5',
#        'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14']) for item in train_df.columns.values])

# D -> Inf
# np.array([('float16' in str(item)) for item in train_df.dtypes.values])*np.array([(str(item) in ['TransactionAmt','dist1', 'dist2','D1',
#        'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9', 'D10', 'D11',
#        'D12', 'D13', 'D14']) for item in train_df.columns.values])

# # V -> Inf
# np.array([('float16' in str(item)) for item in train_df.dtypes.values])*np.array([(str(item) in ['TransactionAmt','dist1', 'dist2','V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7',
#        'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17',
#        'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26',
#        'V27', 'V28', 'V29', 'V30', 'V31', 'V32', 'V33', 'V34', 'V35',
#        'V36', 'V37', 'V38', 'V39', 'V40', 'V41', 'V42', 'V43', 'V44',
#        'V45', 'V46', 'V47', 'V48', 'V49', 'V50', 'V51', 'V52', 'V53',
#        'V54', 'V55', 'V56', 'V57', 'V58', 'V59', 'V60', 'V61', 'V62',
#        'V63', 'V64', 'V65', 'V66', 'V67', 'V68', 'V69', 'V70', 'V71',
#        'V72', 'V73', 'V74', 'V75', 'V76', 'V77', 'V78', 'V79', 'V80',
#        'V81', 'V82', 'V83', 'V84', 'V85', 'V86', 'V87', 'V88', 'V89',
#        'V90', 'V91', 'V92', 'V93', 'V94', 'V95', 'V96', 'V97', 'V98',
#        'V99', 'V100', 'V101', 'V102', 'V103', 'V104', 'V105', 'V106',
#        'V107', 'V108', 'V109', 'V110', 'V111', 'V112', 'V113', 'V114',
#        'V115', 'V116', 'V117', 'V118', 'V119', 'V120', 'V121', 'V122',
#        'V123', 'V124', 'V125', 'V129', 'V130', 'V131', 'V138', 'V139',
#        'V140', 'V141', 'V142', 'V143', 'V144', 'V145', 'V146', 'V147',
#        'V148', 'V149', 'V150', 'V151', 'V152', 'V153', 'V154', 'V155',
#        'V156', 'V157', 'V158', 'V159', 'V161', 'V162', 'V163', 'V167',
#        'V168', 'V169', 'V170', 'V171', 'V172', 'V173', 'V174', 'V175',
#        'V176', 'V177', 'V178', 'V179', 'V180', 'V181', 'V182', 'V183',
#        'V184', 'V185', 'V186', 'V187', 'V188', 'V189', 'V190', 'V191',
#        'V192', 'V193', 'V194', 'V195', 'V196', 'V197', 'V198', 'V199',
#        'V200', 'V201', 'V205', 'V206', 'V207', 'V208', 'V209', 'V210',
#        'V217', 'V218', 'V219', 'V220', 'V221', 'V222', 'V223', 'V224',
#        'V225', 'V226', 'V227', 'V228', 'V229', 'V230', 'V231', 'V232',
#        'V233', 'V234', 'V235', 'V236', 'V237', 'V238', 'V239', 'V240',
#        'V241', 'V242', 'V243', 'V244', 'V245', 'V246', 'V247', 'V248',
#        'V249', 'V250', 'V251', 'V252', 'V253', 'V254', 'V255', 'V256',
#        'V257', 'V258', 'V259', 'V260', 'V261', 'V262', 'V266', 'V267',
#        'V268', 'V269', 'V270', 'V271', 'V272', 'V273', 'V275', 'V279',
#        'V280', 'V281', 'V282', 'V283', 'V284', 'V285', 'V286', 'V287',
#        'V288', 'V289', 'V290', 'V291', 'V292', 'V293', 'V294', 'V295',
#        'V296', 'V297', 'V298', 'V299', 'V300', 'V301', 'V302', 'V303',
#        'V304', 'V305', 'V309', 'V310', 'V311', 'V312', 'V313', 'V314',
#        'V315', 'V322', 'V323', 'V324', 'V325', 'V326', 'V327', 'V328',
#        'V329', 'V330', 'V334', 'V335', 'V336']) for item in train_df.columns.values])

# id -> Inf
# np.array([('float16' in str(item)) for item in train_df.dtypes.values])*np.array([(str(item) in  ['TransactionAmt','dist1', 'dist2','id_01', 'id_03', 'id_04',
#        'id_05', 'id_06', 'id_07', 'id_08', 'id_09', 'id_10', 'id_11']) for item in train_df.columns.values])
# card related -> Inf
# np.array([('float16' in str(item)) for item in train_df.dtypes.values])*np.array([(str(item) in  ['TransactionAmt','dist1', 'dist2','card1_D1_productCD_Amt_mean', 'card1_D1_productCD_Amt_std']) for item in train_df.columns.values])


In [25]:
test_cat = test_df.loc[:,possible_categorical_features_train]
# continuous features
# test = test_df.loc[:,[not item for item in possible_categorical_features_train]]
test = test_df.loc[:,possible_continuous_features_train]


In [26]:
test_cat

,addr1,addr2,P_emaildomain,R_emaildomain,M6,email_type,email_null_concat,m_comb,card1_addr2,TransactionAmt_ProductCD,...,id_30,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo,same_Product_po
0,28,48,16,32,0,2,1,39,366,6297,...,74,190,3,2,2,2,2,2,1187,27087
1,82,48,2,32,0,2,1,194,7754,8297,...,74,190,3,2,2,2,2,2,1187,88429
2,152,48,19,32,0,2,1,32,7884,2875,...,74,190,3,2,2,2,2,2,1187,93783
3,43,48,16,32,1,2,1,15,808,5719,...,74,190,3,2,2,2,2,2,1187,135468
4,67,48,16,32,0,2,1,28,5904,9761,...,74,190,3,2,2,2,2,2,1187,140449
5,175,48,16,32,1,2,1,21,2161,9047,...,74,190,3,2,2,2,2,2,1187,137561
6,5,48,16,32,0,2,1,230,4850,10903,...,74,190,3,2,2,2,2,2,1187,212337
7,38,48,16,32,1,2,1,175,3756,7242,...,74,190,3,2,2,2,2,2,1187,212157
8,164,48,16,32,0,2,1,28,6809,347,...,74,190,3,2,2,2,2,2,1187,98282
9,153,48,32,32,0,0,3,209,1925,928,...,74,190,3,2,2,2,2,2,1187,124847


In [27]:
train_cat

,addr1,addr2,P_emaildomain,R_emaildomain,M6,email_type,email_null_concat,m_comb,card1_addr2,TransactionAmt_ProductCD,...,id_30,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo,same_Product_po
0,91,48,32,32,1,0,3,144,2933,9807,...,74,190,3,2,2,2,2,2,1187,51570
1,96,48,16,32,1,2,1,125,6768,5782,...,74,190,3,2,2,2,2,2,1187,54776
2,98,48,36,32,0,2,1,62,7994,9151,...,74,190,3,2,2,2,2,2,1187,54964
3,153,48,55,32,0,2,1,126,5986,8497,...,74,190,3,2,2,2,2,2,1187,37620
4,128,48,16,32,2,2,1,88,7898,8494,...,8,116,2,1,0,1,1,1,0,54518
5,71,48,16,32,1,2,1,218,8820,8297,...,74,190,3,2,2,2,2,2,1187,54917
6,10,48,55,32,0,2,1,168,1759,2450,...,74,190,3,2,2,2,2,2,1187,54653
7,96,48,29,32,0,2,1,5,2056,7672,...,74,190,3,2,2,2,2,2,1187,54879
8,101,48,1,32,2,2,1,88,6809,2140,...,63,32,1,1,0,0,1,1,1174,54507
9,42,48,55,32,1,2,1,180,5496,928,...,74,190,3,2,2,2,2,2,1187,43866


In [28]:
# train.describe()

In [29]:
len(train_df.columns.values)

520

In [30]:
# train_cat = train_df.loc[:,possible_categorical_features_train]

In [31]:
len(train.columns.values)

491

In [32]:
len(train_cat.columns.values)

29

In [33]:
# train = train_df.loc[:,possible_continuous_features_train]
# test = test_df.loc[:,possible_continuous_features_test]

# del train_df
# del test_df

In [34]:
print(train.columns, test.columns)
continuous_features = list(train.columns.values)

Index(['TransactionAmt', 'card1', 'card2', 'dist1', 'dist2', 'C1', 'C2', 'C3',
       'C4', 'C5',
       ...
       'D_sum', 'D_mean', 'D_std', 'D_min', 'D_max', 'D_na_counts',
       'same_Product_po_cnt', 'same_Product_po_cnt_bydate',
       'account_make_date_D1_count_full', 'TransactionPerHour'],
      dtype='object', length=491) Index(['TransactionAmt', 'card1', 'card2', 'dist1', 'dist2', 'C1', 'C2', 'C3',
       'C4', 'C5',
       ...
       'D_sum', 'D_mean', 'D_std', 'D_min', 'D_max', 'D_na_counts',
       'same_Product_po_cnt', 'same_Product_po_cnt_bydate',
       'account_make_date_D1_count_full', 'TransactionPerHour'],
      dtype='object', length=491)


In [35]:
import numpy as np
import pandas as pd

# train = pd.read_csv('../input/ieee-fraud-detection/train_transaction.csv')
# test = pd.read_csv('../input/ieee-fraud-detection/test_transaction.csv')
sub = pd.read_csv('../input/ieee-fraud-detection/sample_submission.csv')

In [36]:
# train_identity = pd.read_csv('../input/ieee-fraud-detection/train_identity.csv')
# test_identity = pd.read_csv('../input/ieee-fraud-detection/test_identity.csv')

In [37]:
# train = train.merge(train_identity, how='left', on = 'TransactionID')
# test = test.merge(test_identity, how='left', on = 'TransactionID')

In [38]:
# train.iloc[:,-10:]

In [39]:
# train.iloc[:,-28:]

In [40]:
# useful_features = list(train.iloc[:, 3:55].columns+[])

In [41]:
# useful_features = list(train.iloc[:, 3:55].columns)+['id_13','id_31']

# y = train.sort_values('TransactionDT')['isFraud']
# X = train.sort_values('TransactionDT')[useful_features]
# X_test = test[useful_features]
# del train, test

In [42]:
# useful_features

In [43]:
# categorical_features = [
#     'ProductCD',
#     'card1', 'card2', 'card3', 'card4', 'card5', 'card6',
#     'addr1', 'addr2',
#     'P_emaildomain',
#     'R_emaildomain',
#     'M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9'#,'id_31'
# ]

# continuous_features = list(filter(lambda x: x not in categorical_features, X))

In [44]:
train.loc[:,['std' in str(item) for item in train.columns.values]]

,card1_TransactionAmt_std,card2_TransactionAmt_std,card3_TransactionAmt_std,card5_TransactionAmt_std,uid_TransactionAmt_std,uid2_TransactionAmt_std,card1_D1_productCD_Amt_std,D1_productCD_Amt_std,D_std
0,inf,NaN,255.25000,inf,210.75000,176.12500,NaN,179.375000,NaN
1,inf,inf,255.25000,inf,inf,inf,NaN,inf,NaN
2,130.37500,228.625000,255.25000,141.125,130.37500,55.18750,77.062500,inf,NaN
3,196.50000,226.750000,255.25000,191.875,196.75000,211.75000,52.468750,136.750000,NaN
4,69.81250,inf,255.25000,inf,69.81250,NaN,NaN,55.968750,NaN
5,82.93750,214.875000,255.25000,242.500,82.93750,116.25000,0.000000,inf,NaN
6,167.00000,144.625000,255.25000,141.125,167.25000,56.90625,NaN,inf,NaN
7,215.50000,228.625000,255.25000,242.500,215.75000,217.50000,118.812500,inf,NaN
8,inf,inf,255.25000,242.500,inf,221.25000,17.671875,55.968750,NaN
9,190.37500,inf,255.25000,196.500,190.37500,186.25000,129.000000,148.250000,NaN


In [45]:
# continuous_train.loc[:,['std' in str(item) for item in continuous_train.columns.values]]

# Data Processing
For continuous right-skewed features we wil apply log-transform, so that will make them look more like normal distributed.

In [46]:
import scipy
class GaussRankScaler:
    def __init__(self):
        self.epsilon = 1e-9
        self.lower = -1 + self.epsilon
        self.upper = 1 - self.epsilon
        self.range = self.upper - self.lower
        
    def transform(self, X):
        i = np.argsort(X, axis = 0)
        j = np.argsort(i, axis = 0)
        
        assert ( j.min() == 0).all()
        assert ( j.max() == (len(j) - 1 )).all()
        
        j_range = len(j)-1
        self.divider = j_range/ self.range
        
        transformed = j / self.divider
        transformed = transformed - self.upper
        transformed = scipy.special.erfinv( transformed )
        
        return transformed
        

class ContinuousFeatureConverter:
    def __init__(self, name, feature, log_transform):
        self.name = name
        self.skew = feature.skew()
        self.log_transform = log_transform
        
    def transform(self, feature):
        print(self.name, self.skew)
        if self.skew > 1:
            feature = self.log_transform(feature)
        
        mean = feature.mean()
        std = feature.std()
        print(mean,std)
        return (feature - mean)/(std + 1e-6)        

In [47]:
continuous_features

['TransactionAmt',
 'card1',
 'card2',
 'dist1',
 'dist2',
 'C1',
 'C2',
 'C3',
 'C4',
 'C5',
 'C6',
 'C7',
 'C8',
 'C9',
 'C10',
 'C11',
 'C12',
 'C13',
 'C14',
 'D1',
 'D2',
 'D3',
 'D4',
 'D5',
 'D6',
 'D7',
 'D8',
 'D9',
 'D10',
 'D11',
 'D12',
 'D13',
 'D14',
 'V1',
 'V2',
 'V3',
 'V4',
 'V5',
 'V6',
 'V7',
 'V8',
 'V9',
 'V10',
 'V11',
 'V12',
 'V13',
 'V14',
 'V15',
 'V16',
 'V17',
 'V18',
 'V19',
 'V20',
 'V21',
 'V22',
 'V23',
 'V24',
 'V25',
 'V26',
 'V27',
 'V28',
 'V29',
 'V30',
 'V31',
 'V32',
 'V33',
 'V34',
 'V35',
 'V36',
 'V37',
 'V38',
 'V39',
 'V40',
 'V41',
 'V42',
 'V43',
 'V44',
 'V45',
 'V46',
 'V47',
 'V48',
 'V49',
 'V50',
 'V51',
 'V52',
 'V53',
 'V54',
 'V55',
 'V56',
 'V57',
 'V58',
 'V59',
 'V60',
 'V61',
 'V62',
 'V63',
 'V64',
 'V65',
 'V66',
 'V67',
 'V68',
 'V69',
 'V70',
 'V71',
 'V72',
 'V73',
 'V74',
 'V75',
 'V76',
 'V77',
 'V78',
 'V79',
 'V80',
 'V81',
 'V82',
 'V83',
 'V84',
 'V85',
 'V86',
 'V87',
 'V88',
 'V89',
 'V90',
 'V91',
 'V92',
 'V93',


In [48]:
test.columns.values

array(['TransactionAmt', 'card1', 'card2', 'dist1', 'dist2', 'C1', 'C2',
       'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12',
       'C13', 'C14', 'D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9',
       'D10', 'D11', 'D12', 'D13', 'D14', 'V1', 'V2', 'V3', 'V4', 'V5',
       'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15',
       'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24',
       'V25', 'V26', 'V27', 'V28', 'V29', 'V30', 'V31', 'V32', 'V33',
       'V34', 'V35', 'V36', 'V37', 'V38', 'V39', 'V40', 'V41', 'V42',
       'V43', 'V44', 'V45', 'V46', 'V47', 'V48', 'V49', 'V50', 'V51',
       'V52', 'V53', 'V54', 'V55', 'V56', 'V57', 'V58', 'V59', 'V60',
       'V61', 'V62', 'V63', 'V64', 'V65', 'V66', 'V67', 'V68', 'V69',
       'V70', 'V71', 'V72', 'V73', 'V74', 'V75', 'V76', 'V77', 'V78',
       'V79', 'V80', 'V81', 'V82', 'V83', 'V84', 'V85', 'V86', 'V87',
       'V88', 'V89', 'V90', 'V91', 'V92', 'V93', 'V94', 'V95', 'V96',
       

In [49]:
train.columns.values

array(['TransactionAmt', 'card1', 'card2', 'dist1', 'dist2', 'C1', 'C2',
       'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12',
       'C13', 'C14', 'D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9',
       'D10', 'D11', 'D12', 'D13', 'D14', 'V1', 'V2', 'V3', 'V4', 'V5',
       'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15',
       'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24',
       'V25', 'V26', 'V27', 'V28', 'V29', 'V30', 'V31', 'V32', 'V33',
       'V34', 'V35', 'V36', 'V37', 'V38', 'V39', 'V40', 'V41', 'V42',
       'V43', 'V44', 'V45', 'V46', 'V47', 'V48', 'V49', 'V50', 'V51',
       'V52', 'V53', 'V54', 'V55', 'V56', 'V57', 'V58', 'V59', 'V60',
       'V61', 'V62', 'V63', 'V64', 'V65', 'V66', 'V67', 'V68', 'V69',
       'V70', 'V71', 'V72', 'V73', 'V74', 'V75', 'V76', 'V77', 'V78',
       'V79', 'V80', 'V81', 'V82', 'V83', 'V84', 'V85', 'V86', 'V87',
       'V88', 'V89', 'V90', 'V91', 'V92', 'V93', 'V94', 'V95', 'V96',
       

In [50]:
from tqdm.autonotebook import tqdm

feature_converters = {}
continuous_features_processed = []
continuous_features_processed_test = []

for f in tqdm(continuous_features):
    feature = train[f].astype(np.float32)
    feature_test = test[f].astype(np.float32)
    log = lambda x: np.log10(x + 1 - min(0, x.min()))
    converter = ContinuousFeatureConverter(f, feature, log)
#     if feature.min() == -99999:
#         converter = ContinuousFeatureConverter(f, feature, log)
#     else:
#         converter = GaussRankScaler()
    feature_converters[f] = converter
    continuous_features_processed.append(converter.transform(feature))
    continuous_features_processed_test.append(converter.transform(feature_test))
    
continuous_train = pd.DataFrame({s.name: s for s in continuous_features_processed}).astype(np.float32)
continuous_test = pd.DataFrame({s.name: s for s in continuous_features_processed_test}).astype(np.float32)

/home/soouee/anaconda3/envs/tensorflow_vsb/lib/python3.6/site-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


TransactionAmt 14.373494
1.9034873 0.40702173
TransactionAmt 14.373494
1.8917896 0.42013997
card1 -6.211963
7967.812 15246.677
card1 -6.211963
7926.6304 15576.038
card2 -10.627932
-501.94696 9277.761
card2 -10.627932
335.59512 1689.9242
dist1 5.108282
1.1158675 0.7982313
dist1 5.108282
1.0425305 0.7375243
dist2 5.972924
1.5790571 0.93507344
dist2 5.972924
1.6111963 0.90884453
C1 23.957556
0.5291653 0.42239654
C1 23.957556
0.5108794 0.39464778
C2 23.677334
0.53246105 0.42557582
C2 23.677334
0.5198623 0.40175936
C3 88.949745
0.001392107 0.0232511
C3 88.949745
0.0078723375 0.049565367
C4 22.077671
0.09480755 0.24682058
C4 22.077671
0.10594898 0.2393759
C5 5.79367
0.22171034 0.44009247
C5 5.79367
0.19982556 0.41670367
C6 19.766872
0.44469845 0.40217966
C6 19.766872
0.43154132 0.3770562
C7 27.194033
0.04992292 0.20104225
C7 27.194033
0.059305966 0.2028243
C8 26.083181
0.107187636 0.27161282
C8 26.083181
0.1186609 0.25839603
C9 5.69375
0.34938478 0.3971741
C9 5.69375
0.34700665 0.38941684
C1

0.8644285 0.40345955
V83 0.5609962
0.88196486 0.47075674
V83 0.5609962
0.896656 0.45483586
V84 2.7137942
0.04058528 0.104974225
V84 2.7137942
0.07645692 0.13404533
V85 3.7419522
0.04261808 0.11199782
V85 3.7419522
0.07989513 0.14238966
V86 16.930538
0.31074128 0.051958006
V86 16.930538
0.3130239 0.061987888
V87 14.804525
0.31616646 0.062262114
V87 14.804525
0.31868932 0.07267606
V88 -36.378567
0.99924606 0.027447373
V88 -36.378567
0.9998302 0.01303082
V89 38.588844
0.00026390856 0.009187061
V89 38.588844
0.0068250555 0.045014456
V90 0.7424827
0.40186208 0.5161529
V90 0.7424827
0.34095347 0.49428344
V91 1.2462711
0.12263836 0.15614435
V91 1.2462711
0.10426073 0.15015171
V92 2.5271273
0.044520278 0.10901496
V92 2.5271273
0.0771506 0.13402279
V93 3.5891907
0.04516689 0.11142509
V93 3.5891907
0.07822169 0.1367789
V94 2.1119113
0.041242216 0.10351287
V94 2.1119113
0.04596439 0.10828821
V95 30.242123
0.057541247 0.18333817
V95 30.242123
0.05417842 0.14656776
V96 26.02918
0.20309544 0.3279535

0.279072 0.08589271
V198 15.33321
0.28418642 0.085927784
V198 15.33321
0.28255206 0.10217074
V199 13.750156
0.33049163 0.11006548
V199 13.750156
0.33305755 0.12996057
V200 16.575851
0.30334404 0.12249846
V200 16.575851
0.29930857 0.12494836
V201 15.689874
0.3072985 0.12998444
V201 15.689874
0.307315 0.14553288
V202 14.596262
0.51369137 0.98248327
V202 14.596262
0.50448054 0.94871
V203 10.56336
0.65122676 1.094956
V203 10.56336
0.6589502 1.0556878
V204 11.670979
0.5889237 1.046263
V204 11.670979
0.5880847 1.0084699
V205 84.93013
0.13939568 0.5063795
V205 84.93013
0.13104632 0.4845643
V206 207.91333
0.08261422 0.38081902
V206 207.91333
0.08179938 0.377099
V207 21.01544
0.22445714 0.65737766
V207 21.01544
0.21505557 0.6121669
V208 25.523115
0.18926348 0.5451254
V208 25.523115
0.19296594 0.5413539
V209 11.944558
0.22915387 0.6519394
V209 11.944558
0.22518148 0.61409885
V210 14.673164
0.20483172 0.59011847
V210 14.673164
0.20834655 0.5782795
V211 15.000719
0.29536602 0.80750316
V211 15.0007

0.32073444 0.7287816
V313 14.026421
0.31934756 0.7255274
V314 10.748591
0.41052505 0.85382473
V314 10.748591
0.4014716 0.84176785
V315 12.116676
0.3411326 0.76421624
V315 12.116676
0.3385998 0.76028925
V316 30.09739
0.16585198 0.6095192
V316 30.09739
0.14981666 0.5687193
V317 24.597406
0.2852343 0.8061256
V317 24.597406
0.24296466 0.73269564
V318 25.482828
0.21659188 0.7029797
V318 25.482828
0.19121446 0.6485321
V319 181.83365
0.102962 0.4788155
V319 181.83365
0.12134468 0.515568
V320 70.51556
0.17814445 0.6244152
V320 70.51556
0.2059729 0.6575168
V321 123.55647
0.13065612 0.54063696
V321 123.55647
0.15075758 0.5721231
V322 11.19745
0.1170409 0.34891778
V322 11.19745
0.081305146 0.19461998
V323 9.734561
0.17460722 0.43346828
V323 9.734561
0.13735798 0.2933961
V324 9.363781
0.14583035 0.39199913
V324 9.363781
0.10775878 0.23675273
V325 8.357877
0.015801815 0.07492083
V325 8.357877
0.012885149 0.06793414
V326 5.8000593
0.07959492 0.26391512
V326 5.8000593
0.06118445 0.20842417
V327 5.862

266273.03 50535.082
id_26_count_full -4.997382
266080.1 51121.246
id_26_count_full -4.997382
266143.56 50962.098
id_27_count_full -4.9941454
266386.06 49394.31
id_27_count_full -4.9941454
266431.1 49279.547
id_28_count_full -3.690197
136963.16 23327.512
id_28_count_full -3.690197
135472.64 27661.346
id_29_count_full -4.381737
136572.05 22009.5
id_29_count_full -4.381737
135032.28 26545.436
id_30_count_full 0.032495752
72750.73 61743.566
id_30_count_full 0.032495752
77999.59 61423.62
id_32_count_full -1.0087724
109459.805 33396.875
id_32_count_full -1.0087724
112687.35 31655.584
id_33_count_full -0.016160464
76605.76 65083.117
id_33_count_full -0.016160464
77687.77 65347.82
id_34_count_full -2.2827122
119802.71 38629.164
id_34_count_full -2.2827122
134137.47 2011.2329
id_35_count_full -4.3916245
137099.69 22117.229
id_35_count_full -4.3916245
134903.84 26150.656
id_36_count_full -3.3560808
249200.36 65918.77
id_36_count_full -3.3560808
251672.84 61658.94
id_37_count_full -1.3112918
1782

In [51]:
train[f].min()

0.00026185844680532696

In [52]:
# train.uid2_TransactionAmt_std

In [53]:
continuous_train['isna_sum'] = continuous_train.isna().sum(axis=1)
continuous_test['isna_sum'] = continuous_test.isna().sum(axis=1)

continuous_train['isna_sum'] = (continuous_train['isna_sum'] - continuous_train['isna_sum'].mean())/continuous_train['isna_sum'].std()
continuous_test['isna_sum'] = (continuous_test['isna_sum'] - continuous_test['isna_sum'].mean())/continuous_test['isna_sum'].std()

In [54]:
# continuous_train.uid2_TransactionAmt_std

In [55]:
# continuous_train.loc[:,(continuous_train.min() == continuous_train.max()).values]

In [56]:
continuous_train.V1

0         0.007387
1              NaN
2         0.007387
3              NaN
4              NaN
5         0.007387
6         0.007387
7              NaN
8              NaN
9         0.007387
10             NaN
11             NaN
12             NaN
13             NaN
14             NaN
15        0.007387
16             NaN
17             NaN
18        0.007387
19             NaN
20        0.007387
21             NaN
22             NaN
23        0.007387
24             NaN
25             NaN
26             NaN
27        0.007387
28             NaN
29             NaN
            ...   
590510    0.007387
590511    0.007387
590512    0.007387
590513    0.007387
590514    0.007387
590515    0.007387
590516    0.007387
590517    0.007387
590518    0.007387
590519    0.007387
590520    0.007387
590521         NaN
590522    0.007387
590523         NaN
590524    0.007387
590525    0.007387
590526         NaN
590527    0.007387
590528    0.007387
590529         NaN
590530         NaN
590531      

In [57]:
from scipy import stats

isna_columns = []
for column in tqdm(continuous_features):
    isna = continuous_train[column].isna()
    if isna.mean() > 0.:
        continuous_train[column + '_isna'] = isna.astype(int)
        continuous_test[column + '_isna'] = continuous_test[column].isna().astype(int)
        isna_columns.append(column)
        
continuous_train = continuous_train.fillna(0.)
continuous_test = continuous_test.fillna(0.)


In [58]:
continuous_train.shape

(590540, 922)

In [59]:
from sklearn.decomposition import PCA

pca = PCA(n_components=11).fit(continuous_train.loc[:,np.array(['V' in item for item in continuous_train.columns.values])*\
                    np.array([(not ('isna' in item)) for item in continuous_train.columns.values])])
def convert_pca(target, pca):
    pca_result = pca.transform(target.loc[:,np.array(['V' in item for item in target.columns.values])*\
                        np.array([(not ('isna' in item)) for item in target.columns.values])])
    pca_result_df = pd.DataFrame(data=pca_result, columns=['pca{}'.format(item) for item in range(11)])
    v_column_names = target.columns.values[np.array(['V' in item for item in target.columns.values])*\
                        np.array([(not ('isna' in item)) for item in target.columns.values])]

    target = target.drop(columns=v_column_names)
    target.columns.values[np.array(['V' in item for item in target.columns.values])*\
                        np.array([(not ('isna' in item)) for item in target.columns.values])]
    target = pd.concat([target,pca_result_df],axis = 1)
    return target

In [60]:
continuous_train = convert_pca(continuous_train,pca)
continuous_test = convert_pca(continuous_test,pca)

In [61]:
continuous_train.loc[:,np.array(['V' in item for item in continuous_train.columns.values])*\
                    np.array([(not ('isna' in item)) for item in continuous_train.columns.values])].columns.values

array([], dtype=object)

In [62]:
# continuous_train.shape

For categorical features we will apply OneHot transformation, but only for most common values for each feature to reduce sparsity. <br>
Also there is an embedding approach for categorical features transformation. It was implemented in this kernel https://www.kaggle.com/ryches/keras-nn-starter-w-time-series-split <br>
With embedding approach I didn't get any significant improvement comparing to this.

In [63]:
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from tqdm.autonotebook import tqdm

def categorical_encode(df_train, df_test, categorical_features, n_values=50):
    df_train = df_train[categorical_features].astype(str)
    df_test = df_test[categorical_features].astype(str)
    
    categories = []
    for column in tqdm(categorical_features):
        if False:
            categories.append(list(df_train[column].value_counts().iloc[: n_values - 1].index) + ['Other'])
            values2use = categories[-1]
            df_train[column] = df_train[column].apply(lambda x: x if x in values2use else 'Other')
            df_test[column] = df_test[column].apply(lambda x: x if x in values2use else 'Other')
    if False:
        ohe = OneHotEncoder(categories=categories)
        ohe.fit(pd.concat([df_train, df_test]))
        df_train = pd.DataFrame(ohe.transform(df_train).toarray()).astype(np.float16)
        df_test = pd.DataFrame(ohe.transform(df_test).toarray()).astype(np.float16)
    else:
        ohe = OrdinalEncoder()
        ohe.fit(pd.concat([df_train, df_test]))
        df_train = pd.DataFrame(ohe.transform(df_train),columns=df_train.columns.values).astype(np.float32)
        df_test = pd.DataFrame(ohe.transform(df_test),columns=df_train.columns.values).astype(np.float32)
    
    return df_train, df_test

In [64]:
train_cat['P_emaildomain'].value_counts()

16    228355
55    100934
32     94456
19     45250
1      36998
2      28289
9       7888
23      6267
36      5096
31      4092
3       4033
25      3041
44      2970
50      2705
60      2396
4       1909
56      1543
30      1522
10      1393
35      1011
8        816
26       749
43       664
29       559
11       514
15       496
37       438
28       436
24       322
0        315
42       305
21       305
52       305
22       295
13       280
12       260
51       240
49       230
33       230
38       207
6        205
34       196
14       195
41       189
48       175
7        172
45       164
5        159
17       149
59       143
58       134
18       112
39        76
57        74
40        68
27        56
54        49
20        43
47        35
53        32
Name: P_emaildomain, dtype: int64

In [65]:
categorical_features = list(train_cat.columns.values)

In [66]:
train_cat.columns.values

array(['addr1', 'addr2', 'P_emaildomain', 'R_emaildomain', 'M6',
       'email_type', 'email_null_concat', 'm_comb', 'card1_addr2',
       'TransactionAmt_ProductCD', 'addr1_addr2', 'id_12', 'id_14',
       'id_15', 'id_16', 'id_23', 'id_27', 'id_28', 'id_29', 'id_30',
       'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38', 'DeviceType',
       'DeviceInfo', 'same_Product_po'], dtype=object)

In [67]:
train_categorical, test_categorical = categorical_encode(train_cat, test_cat, categorical_features)

In [68]:
train_categorical

,addr1,addr2,P_emaildomain,R_emaildomain,M6,email_type,email_null_concat,m_comb,card1_addr2,TransactionAmt_ProductCD,...,id_30,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo,same_Product_po
0,176.0,43.0,26.0,26.0,1.0,0.0,3.0,51.0,3711.0,11368.0,...,72.0,102.0,3.0,2.0,2.0,2.0,2.0,2.0,210.0,158569.0
1,181.0,43.0,8.0,26.0,1.0,2.0,1.0,30.0,7971.0,6896.0,...,72.0,102.0,3.0,2.0,2.0,2.0,2.0,2.0,210.0,162130.0
2,183.0,43.0,30.0,26.0,0.0,2.0,1.0,215.0,9333.0,10640.0,...,72.0,102.0,3.0,2.0,2.0,2.0,2.0,2.0,210.0,162339.0
3,61.0,43.0,51.0,26.0,0.0,2.0,1.0,31.0,7102.0,9912.0,...,72.0,102.0,3.0,2.0,2.0,2.0,2.0,2.0,210.0,143069.0
4,33.0,43.0,8.0,26.0,2.0,2.0,1.0,243.0,9226.0,9909.0,...,74.0,20.0,2.0,1.0,0.0,1.0,1.0,1.0,0.0,161844.0
5,154.0,43.0,8.0,26.0,1.0,2.0,1.0,133.0,10252.0,9690.0,...,72.0,102.0,3.0,2.0,2.0,2.0,2.0,2.0,210.0,162287.0
6,2.0,43.0,51.0,26.0,0.0,2.0,1.0,77.0,2406.0,3195.0,...,72.0,102.0,3.0,2.0,2.0,2.0,2.0,2.0,210.0,161994.0
7,181.0,43.0,22.0,26.0,0.0,2.0,1.0,201.0,2737.0,8996.0,...,72.0,102.0,3.0,2.0,2.0,2.0,2.0,2.0,210.0,162244.0
8,4.0,43.0,1.0,26.0,2.0,2.0,1.0,243.0,8017.0,2851.0,...,60.0,117.0,1.0,1.0,0.0,0.0,1.0,1.0,196.0,161832.0
9,122.0,43.0,51.0,26.0,1.0,2.0,1.0,91.0,6558.0,10782.0,...,72.0,102.0,3.0,2.0,2.0,2.0,2.0,2.0,210.0,150008.0


In [69]:
test_categorical

,addr1,addr2,P_emaildomain,R_emaildomain,M6,email_type,email_null_concat,m_comb,card1_addr2,TransactionAmt_ProductCD,...,id_30,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo,same_Product_po
0,106.0,43.0,8.0,26.0,0.0,2.0,1.0,189.0,4518.0,7468.0,...,72.0,102.0,3.0,2.0,2.0,2.0,2.0,2.0,210.0,131365.0
1,166.0,43.0,12.0,26.0,0.0,2.0,1.0,106.0,9067.0,9690.0,...,72.0,102.0,3.0,2.0,2.0,2.0,2.0,2.0,210.0,199522.0
2,60.0,43.0,11.0,26.0,0.0,2.0,1.0,182.0,9211.0,3666.0,...,72.0,102.0,3.0,2.0,2.0,2.0,2.0,2.0,210.0,205471.0
3,123.0,43.0,8.0,26.0,1.0,2.0,1.0,57.0,9429.0,6826.0,...,72.0,102.0,3.0,2.0,2.0,2.0,2.0,2.0,210.0,39412.0
4,149.0,43.0,8.0,26.0,0.0,2.0,1.0,177.0,7012.0,11317.0,...,72.0,102.0,3.0,2.0,2.0,2.0,2.0,2.0,210.0,44947.0
5,85.0,43.0,8.0,26.0,1.0,2.0,1.0,124.0,2854.0,10524.0,...,72.0,102.0,3.0,2.0,2.0,2.0,2.0,2.0,210.0,41738.0
6,130.0,43.0,8.0,26.0,0.0,2.0,1.0,147.0,5841.0,1007.0,...,72.0,102.0,3.0,2.0,2.0,2.0,2.0,2.0,210.0,124823.0
7,117.0,43.0,8.0,26.0,1.0,2.0,1.0,85.0,4625.0,8519.0,...,72.0,102.0,3.0,2.0,2.0,2.0,2.0,2.0,210.0,124623.0
8,73.0,43.0,8.0,26.0,0.0,2.0,1.0,177.0,8017.0,4327.0,...,72.0,102.0,3.0,2.0,2.0,2.0,2.0,2.0,210.0,210470.0
9,61.0,43.0,26.0,26.0,0.0,0.0,3.0,123.0,2591.0,10782.0,...,72.0,102.0,3.0,2.0,2.0,2.0,2.0,2.0,210.0,27611.0


In [70]:
train_categorical.columns.values

array(['addr1', 'addr2', 'P_emaildomain', 'R_emaildomain', 'M6',
       'email_type', 'email_null_concat', 'm_comb', 'card1_addr2',
       'TransactionAmt_ProductCD', 'addr1_addr2', 'id_12', 'id_14',
       'id_15', 'id_16', 'id_23', 'id_27', 'id_28', 'id_29', 'id_30',
       'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38', 'DeviceType',
       'DeviceInfo', 'same_Product_po'], dtype=object)

In [71]:
if True:
    X = pd.concat([continuous_train, train_categorical], axis=1)
#     del continuous_train, train_categorical
    X_test = pd.concat([continuous_test, test_categorical], axis=1)
#     del continuous_test, test_categorical
else:
    X = continuous_train
#     del continuous_train
    X_test = continuous_test
#     del continuous_test

In [72]:
X_test.columns.values

array(['TransactionAmt', 'card1', 'card2', 'dist1', 'dist2', 'C1', 'C2',
       'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12',
       'C13', 'C14', 'D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9',
       'D10', 'D11', 'D12', 'D13', 'D14', 'm_null_sum', 'm_T_sum',
       'account_make_date_D2', 'account_make_date_D10',
       'account_make_date_D15', 'TransactionAmt_decimal_count',
       'TransactionAmt_decimal', 'ProductCD_count_full',
       'card1_count_full', 'card2_count_full', 'card3_count_full',
       'card4_count_full', 'card5_count_full', 'card6_count_full',
       'addr1_count_full', 'addr2_count_full', 'P_emaildomain_count_full',
       'R_emaildomain_count_full', 'P_emaildomain_bin_count_full',
       'R_emaildomain_bin_count_full', 'M1_count_full', 'M2_count_full',
       'M3_count_full', 'M4_count_full', 'M5_count_full', 'M6_count_full',
       'M7_count_full', 'M8_count_full', 'M9_count_full',
       'email_null_concat_count_full', 'card1_addr1_coun

In [73]:
X_test.describe()

,TransactionAmt,card1,card2,dist1,dist2,C1,C2,C3,C4,C5,...,id_30,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo,same_Product_po
count,5.066910e+05,5.066910e+05,5.066910e+05,5.066910e+05,5.066910e+05,5.066910e+05,5.066910e+05,5.066910e+05,5.066910e+05,5.066910e+05,...,506691.000000,506691.000000,506691.000000,506691.000000,506691.000000,506691.000000,506691.000000,506691.000000,506691.000000,506691.000000
mean,-7.233523e-07,-9.826389e-08,5.637450e-08,-8.542002e-08,-4.200984e-09,-2.692545e-07,1.055968e-07,2.192342e-08,2.529625e-08,7.902067e-08,...,65.905914,103.096046,2.857554,1.600735,1.466610,1.665956,1.542058,1.582913,220.463409,90882.671875
std,9.999976e-01,1.000000e+00,9.914234e-01,6.521158e-01,2.681561e-01,9.999945e-01,9.999945e-01,9.999768e-01,9.999928e-01,9.999946e-01,...,17.226027,23.408930,0.349513,0.705486,0.880338,0.591502,0.789584,0.732672,137.052017,42641.800781
min,-4.484303e+00,-6.928953e+00,-5.937225e+01,-1.413552e+00,-1.772794e+00,-1.294517e+00,-1.293961e+00,-1.588242e-01,-4.426032e-01,-4.795376e-01,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000
25%,-6.640763e-01,-1.315245e-01,-7.136126e-02,-1.542798e-03,0.000000e+00,-5.317371e-01,-5.446836e-01,-1.588242e-01,-4.426032e-01,-4.795376e-01,...,72.000000,102.000000,3.000000,1.000000,0.000000,1.000000,1.000000,1.000000,210.000000,59653.000000
50%,-1.269431e-01,1.095509e-01,1.503315e-02,0.000000e+00,0.000000e+00,-5.317371e-01,-5.446836e-01,-1.588242e-01,-4.426032e-01,-4.795376e-01,...,72.000000,102.000000,3.000000,2.000000,2.000000,2.000000,2.000000,2.000000,210.000000,86989.000000
75%,4.964544e-01,3.981352e-01,1.043863e-01,0.000000e+00,0.000000e+00,2.310424e-01,2.045939e-01,-1.588242e-01,8.149534e-01,2.428685e-01,...,72.000000,102.000000,3.000000,2.000000,2.000000,2.000000,2.000000,2.000000,210.000000,116527.000000
max,5.045706e+00,6.721458e-01,1.564596e-01,3.884524e+00,2.589432e+00,7.498033e+00,7.456221e+00,3.020753e+01,1.294491e+01,5.703118e+00,...,75.000000,190.000000,3.000000,2.000000,2.000000,2.000000,2.000000,2.000000,1214.000000,212373.000000


In [74]:
print(len(y),len(X))

590540 590540


In [75]:
# X_tr.describe()

# Validation
For validation I use time-based holdout. For these and other models it has a good correlation between val and lb.

In [76]:
np.random.seed(42) # NumPy
random.seed(42) # Python
tf.set_random_seed(42) # Tensorflow

# Modeling

In [77]:
import keras
import random
import tensorflow as tf
import keras.backend as K

from keras.models import Model
from keras.layers import Dense, Input, Dropout, BatchNormalization, Activation
from keras.utils.generic_utils import get_custom_objects
from keras.optimizers import Adam, Nadam
from keras.callbacks import Callback
from sklearn.metrics import roc_auc_score




Using TensorFlow backend.


In [78]:
# def roc(y_val,y_pred_val):
#     roc = roc_auc_score(y_val, y_pred_val)
# #     print('\rroc-auc_val: %s' % (str(round(roc_val,4))),end=100*' '+'\n')
#     return roc

# Compatible with tensorflow backend
class roc_callback(Callback):
    def __init__(self,
                 #training_data,
                 validation_data,mode='bce'):
        #self.x = training_data[0]
        #self.y = training_data[1]
        self.x_val = validation_data[0]
        self.y_val = validation_data[1]
        self.roc_best = 0.0
        self.mode = mode


    def on_train_begin(self, logs={}):
        return

    def on_train_end(self, logs={}):
        return

    def on_epoch_begin(self, epoch, logs={}):
        return

    def on_epoch_end(self, epoch, logs={}):
        y_pred_val = self.model.predict(self.x_val)
        roc_val = roc_auc_score(self.y_val, y_pred_val)
        print('\rroc-auc_val: %s' % (str(round(roc_val,4))),end=100*' '+'\n')
        if self.roc_best < roc_val:
            print('\rbest model improved {} over {} \n'.format(roc_val,self.roc_best,))
            self.roc_best = roc_val
            self.model.save("downsampling_catcoldiffall_focalg3a0.2_addr1_addr2_real31cat_{}_hlayer1{}_hlayer2{}_dr1{}_dr2{}_fold{}.h5".format(self.mode,hlayer1,hlayer2,dr1,dr2,fold))
        return

    def on_batch_begin(self, batch, logs={}):
        return

    def on_batch_end(self, batch, logs={}):
        return
def focal_loss(gamma=2., alpha=.25):
    def focal_loss_fixed(y_true, y_pred):
        pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
        pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))
        return -K.mean(alpha * K.pow(1. - pt_1, gamma) * K.log(K.epsilon()+pt_1))-K.mean((1-alpha) * K.pow( pt_0, gamma) * K.log(1. - pt_0 + K.epsilon()))
    return focal_loss_fixed

def custom_gelu(x):
    return 0.5 * x * (1 + tf.tanh(tf.sqrt(2 / np.pi) * (x + 0.044715 * tf.pow(x, 3))))

get_custom_objects().update({'custom_gelu': Activation(custom_gelu)})
get_custom_objects().update({'focal_loss_fn': focal_loss(gamma=3.,alpha=0.2)})

In [79]:
class RAdam(keras.optimizers.Optimizer):
    """RAdam optimizer.
    # Arguments
        lr: float >= 0. Learning rate.
        beta_1: float, 0 < beta < 1. Generally close to 1.
        beta_2: float, 0 < beta < 1. Generally close to 1.
        epsilon: float >= 0. Fuzz factor. If `None`, defaults to `K.epsilon()`.
        decay: float >= 0. Learning rate decay over each update.
        weight_decay: float >= 0. Weight decay for each param.
        amsgrad: boolean. Whether to apply the AMSGrad variant of this
            algorithm from the paper "On the Convergence of Adam and
            Beyond".
        total_steps: int >= 0. Total number of training steps. Enable warmup by setting a positive value.
        warmup_proportion: 0 < warmup_proportion < 1. The proportion of increasing steps.
        min_lr: float >= 0. Minimum learning rate after warmup.
    # References
        - [Adam - A Method for Stochastic Optimization](https://arxiv.org/abs/1412.6980v8)
        - [On the Convergence of Adam and Beyond](https://openreview.net/forum?id=ryQu7f-RZ)
        - [On The Variance Of The Adaptive Learning Rate And Beyond](https://arxiv.org/pdf/1908.03265v1.pdf)
    """

    def __init__(self, lr=0.001, beta_1=0.9, beta_2=0.999,
                 epsilon=None, decay=0., weight_decay=0., amsgrad=False,
                 total_steps=0, warmup_proportion=0.1, min_lr=0., **kwargs):
        super(RAdam, self).__init__(**kwargs)
        with K.name_scope(self.__class__.__name__):
            self.iterations = K.variable(0, dtype='int64', name='iterations')
            self.lr = K.variable(lr, name='lr')
            self.beta_1 = K.variable(beta_1, name='beta_1')
            self.beta_2 = K.variable(beta_2, name='beta_2')
            self.decay = K.variable(decay, name='decay')
            self.weight_decay = K.variable(weight_decay, name='weight_decay')
            self.total_steps = K.variable(total_steps, name='total_steps')
            self.warmup_proportion = K.variable(warmup_proportion, name='warmup_proportion')
            self.min_lr = K.variable(lr, name='min_lr')
        if epsilon is None:
            epsilon = K.epsilon()
        self.epsilon = epsilon
        self.initial_decay = decay
        self.initial_weight_decay = weight_decay
        self.initial_total_steps = total_steps
        self.amsgrad = amsgrad

    def get_updates(self, loss, params):
        grads = self.get_gradients(loss, params)
        self.updates = [K.update_add(self.iterations, 1)]

        lr = self.lr

        if self.initial_decay > 0:
            lr = lr * (1. / (1. + self.decay * K.mcp_save_focalcast(self.iterations, K.dtype(self.decay))))

        t = K.cast(self.iterations, K.floatx()) + 1

        if self.initial_total_steps > 0:
            warmup_steps = self.total_steps * self.warmup_proportion
            decay_steps = self.total_steps - warmup_steps
            lr = K.switch(
                t <= warmup_steps,
                lr * (t / warmup_steps),
                lr * (1.0 - K.minimum(t, decay_steps) / decay_steps),
            )

        ms = [K.zeros(K.int_shape(p), dtype=K.dtype(p), name='m_' + str(i)) for (i, p) in enumerate(params)]
        vs = [K.zeros(K.int_shape(p), dtype=K.dtype(p), name='v_' + str(i)) for (i, p) in enumerate(params)]

        if self.amsgrad:
            vhats = [K.zeros(K.int_shape(p), dtype=K.dtype(p), name='vhat_' + str(i)) for (i, p) in enumerate(params)]
        else:
            vhats = [K.zeros(1, name='vhat_' + str(i)) for i in range(len(params))]

        self.weights = [self.iterations] + ms + vs + vhats

        beta_1_t = K.pow(self.beta_1, t)
        beta_2_t = K.pow(self.beta_2, t)

        sma_inf = 2.0 / (1.0 - self.beta_2) - 1.0
        sma_t = sma_inf - 2.0 * t * beta_2_t / (1.0 - beta_2_t)

        for p, g, m, v, vhat in zip(params, grads, ms, vs, vhats):
            m_t = (self.beta_1 * m) + (1. - self.beta_1) * g
            v_t = (self.beta_2 * v) + (1. - self.beta_2) * K.square(g)

            m_corr_t = m_t / (1.0 - beta_1_t)
            if self.amsgrad:
                vhat_t = K.maximum(vhat, v_t)
                v_corr_t = K.sqrt(vhat_t / (1.0 - beta_2_t) + self.epsilon)
                self.updates.append(K.update(vhat, vhat_t))
            else:
                v_corr_t = K.sqrt(v_t / (1.0 - beta_2_t) + self.epsilon)

            r_t = K.sqrt((sma_t - 4.0) / (sma_inf - 4.0) *
                         (sma_t - 2.0) / (sma_inf - 2.0) *
                         sma_inf / sma_t)

            p_t = K.switch(sma_t > 5, r_t * m_corr_t / v_corr_t, m_corr_t)

            if self.initial_weight_decay > 0:
                p_t += self.weight_decay * p

            p_t = p - lr * p_t

            self.updates.append(K.update(m, m_t))
            self.updates.append(K.update(v, v_t))
            new_p = p_t

            # Apply constraints.
            if getattr(p, 'constraint', None) is not None:
                new_p = p.constraint(new_p)

            self.updates.append(K.update(p, new_p))
        return self.updates

    def get_config(self):
        config = {
            'lr': float(K.get_value(self.lr)),
            'beta_1': float(K.get_value(self.beta_1)),
            'beta_2': float(K.get_value(self.beta_2)),
            'decay': float(K.get_value(self.decay)),
            'weight_decay': float(K.get_value(self.weight_decay)),
            'epsilon': self.epsilon,
            'amsgrad': self.amsgrad,
            'total_steps': float(K.get_value(self.total_steps)),
            'warmup_proportion': float(K.get_value(self.warmup_proportion)),
            'min_lr': float(K.get_value(self.min_lr)),
        }
        base_config = super(RAdam, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [80]:
def binary_crossentropy_with_ranking(y_true, y_pred):
    """ Trying to combine ranking loss with numeric precision"""
    # first get the log loss like normal
    logloss = K.mean(K.binary_crossentropy(y_pred, y_true), axis=-1)
    
    # next, build a rank loss
    
    # clip the probabilities to keep stability
    y_pred_clipped = K.clip(y_pred, K.epsilon(), 1-K.epsilon())

    # translate into the raw scores before the logit
    y_pred_score = K.log(y_pred_clipped / (1 - y_pred_clipped))

    # determine what the maximum score for a zero outcome is
    y_pred_score_zerooutcome_max = K.max(y_pred_score * tf.to_float((y_true <1)))

    # determine how much each score is above or below it
    rankloss = y_pred_score - y_pred_score_zerooutcome_max

    # only keep losses for positive outcomes
    rankloss = rankloss * y_true

    # only keep losses where the score is below the max
    rankloss = K.square(K.clip(rankloss, -100, 0))

    # average the loss for just the positive outcomes
    rankloss = K.sum(rankloss, axis=-1) / (K.sum(tf.to_float(y_true > 0)) + 1)

    # return (rankloss + 1) * logloss - an alternative to try
    return 0.5*rankloss + 1*logloss

hlayer1 = 768
hlayer2 = 256
dr1 = 0.3
dr2 = 0.2

In [81]:
train_df

,TransactionAmt,card1,card2,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,...,D_mean,D_std,D_min,D_max,D_na_counts,same_Product_po,same_Product_po_cnt,same_Product_po_cnt_bydate,account_make_date_D1_count_full,TransactionPerHour
0,68.500000,13926.0,NaN,91,48,19.0,NaN,32,32,1.0,...,NaN,NaN,NaN,NaN,5,51570,3,1,3856,0.006033
1,29.000000,2755.0,404.0,96,48,NaN,NaN,16,32,1.0,...,NaN,NaN,NaN,NaN,5,54776,89,34,4297,0.006033
2,59.000000,4663.0,490.0,98,48,287.0,NaN,36,32,1.0,...,NaN,NaN,NaN,NaN,5,54964,259,110,4297,0.006033
3,50.000000,18132.0,567.0,153,48,NaN,NaN,55,32,2.0,...,NaN,NaN,NaN,NaN,5,37620,19,3,1621,0.006033
4,50.000000,4497.0,514.0,128,48,NaN,NaN,16,32,1.0,...,NaN,NaN,NaN,NaN,5,54518,100,94,4297,0.006033
5,49.000000,5937.0,555.0,71,48,36.0,NaN,16,32,1.0,...,NaN,NaN,NaN,NaN,5,54917,137,52,4297,0.006033
6,159.000000,12308.0,360.0,10,48,0.0,NaN,55,32,1.0,...,NaN,NaN,NaN,NaN,5,54653,4,4,4297,0.006033
7,422.500000,12695.0,490.0,96,48,NaN,NaN,29,32,1.0,...,NaN,NaN,NaN,NaN,5,54879,7,5,4297,0.006033
8,15.000000,2803.0,100.0,101,48,NaN,NaN,1,32,1.0,...,NaN,NaN,NaN,NaN,5,54507,13,13,4297,0.006033
9,117.000000,17399.0,111.0,42,48,19.0,NaN,55,32,2.0,...,NaN,NaN,NaN,NaN,5,43866,71,2,880,0.006033


In [82]:


temp_train = pickle.load(open('./train_df_9551ka','rb'))[['year','month']]
train_group = (temp_train['year']-2017)*12 + temp_train['month'] 

In [83]:
# train_categorical.columns.values

In [100]:
numcols = X.loc[:,[(item not in list(train_categorical.columns.values)+['id_01', 'id_02', 'id_03', 'id_04', 'id_05', 'id_06', 'id_07',
       'id_08', 'id_09', 'id_10', 'id_11', 'id_14', 'id_17', 'id_18',
       'id_19', 'id_20', 'id_21', 'id_22', 'id_24', 'id_25', 'id_26',
       'id_32','id_12_count_full', 'id_14_count_full', 'id_15_count_full',
       'id_16_count_full', 'id_17_count_full', 'id_18_count_full',
       'id_19_count_full', 'id_20_count_full', 'id_21_count_full',
       'id_22_count_full', 'id_23_count_full', 'id_24_count_full',
       'id_25_count_full', 'id_26_count_full', 'id_27_count_full',
       'id_28_count_full', 'id_29_count_full', 'id_30_count_full',
       'id_32_count_full', 'id_33_count_full', 'id_34_count_full',
       'id_35_count_full', 'id_36_count_full', 'id_37_count_full',
       'id_38_count_full', 'id_30_31_33_Type_Info_prev_click',
       'id_30_31_33_Type_Info_next_click','card1','card2','id_30_31_33_Type_Info_prev_click',
'id_30_31_33_Type_Info_next_click',
'uid2_next_click',
                                                            'account_make_date_D2',
 'account_make_date_D10',
 'account_make_date_D15',
 'account_make_date_D1_count_full',
                                                                       'D1',
 'D2',
 'D3',
 'D4',
 'D5',
 'D6',
 'D7',
 'D8',
 'D9',
 'D10',
 'D11',
 'D12',
 'D13',
 'D14']) for item in X.columns.values] ].columns.values

catcols = train_categorical.columns.values
new_catcols = []
for c in list(catcols):
    num_unique_values = int(X[c].nunique())

    embed_dim = int(min(np.ceil((num_unique_values)/2),50))
    print(num_unique_values,embed_dim)
    if embed_dim<=31 or c in ['addr1_addr2','m_comb','addr1']:
        new_catcols.append(c)
catcols = new_catcols

185 50
55 28
60 30
61 31
3 2
4 2
4 2
256 50
10814 50
9970 50
301 50
3 2
26 13
4 2
3 2
4 2
3 2
3 2
3 2
76 38
191 50
4 2
3 2
3 2
3 2
3 2
3 2
1215 50
125214 50


In [101]:
train_categorical.columns.values

array(['addr1', 'addr2', 'P_emaildomain', 'R_emaildomain', 'M6',
       'email_type', 'email_null_concat', 'm_comb', 'card1_addr2',
       'TransactionAmt_ProductCD', 'addr1_addr2', 'id_12', 'id_14',
       'id_15', 'id_16', 'id_23', 'id_27', 'id_28', 'id_29', 'id_30',
       'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38', 'DeviceType',
       'DeviceInfo', 'same_Product_po'], dtype=object)

In [102]:
len(catcols), len(numcols)

(23, 525)

In [103]:
# ['addr1','addr2', 'email_type', 'm_comb']

In [104]:
# def create_model(loss_fn):
from keras import layers
from keras import regularizers
from keras.models import load_model
from keras.models import load_model
from sklearn.metrics import confusion_matrix,cohen_kappa_score,recall_score,f1_score,roc_curve
max_embed = 50
def create_model(loss_fn,  mode ='ann'):
    if mode == 'ann':
        inputs = []
        outputs = []
        for c in list(catcols):
            num_unique_values = int(X_tr[c].nunique())
            embed_dim = int(min(np.ceil((num_unique_values)/2),max_embed))
            inp = layers.Input(shape=(1,))
            out = layers.Embedding(num_unique_values+1, embed_dim, name=c)(inp)
            out = layers.SpatialDropout1D(0.3)(out)
            out = layers.Reshape(target_shape=(embed_dim,))(out)
            inputs.append(inp)
            outputs.append(out)
        print('after cat',len(inputs))
        num_input = layers.Input(shape=(X_tr[numcols].shape[1],))
        inputs.append(num_input)
        outputs.append(num_input)
        print('after num',len(inputs))
        x = layers.Concatenate()(outputs)
        x = BatchNormalization()(x)
        x = Dropout(dr1)(x)
        x = Dense(hlayer1, activation=custom_gelu)(x)
        x = BatchNormalization()(x)
        x = Dropout(dr1)(x)
        x = Dense(hlayer2, activation=custom_gelu)(x)
        x = BatchNormalization()(x)
        x = Dropout(dr2)(x)
        x = Dense(1, activation='sigmoid')(x)
        model = Model(inputs=inputs, outputs=x)
        if loss_fn=='binary_crossentropy':
        
            model.compile(
                optimizer=Nadam(),
                loss=[loss_fn]
            )
        elif loss_fn =='binary_crossentropy_with_ranking':
            model.compile(
                optimizer=RAdam(),
                loss=[binary_crossentropy_with_ranking]
            )
        elif loss_fn =='binary_crossentropy_with_ranking':
            model.compile(
                optimizer=RAdam(),
                loss=[binary_crossentropy_with_ranking]
            )
        else:
            model.compile(
                optimizer=Nadam(),
                loss=[loss_fn]
            )
        #model.summary()
        return model
    elif mode == 'autoencoder':
        input_dim = X_tr.shape[1]
        inps = Input(shape=(input_dim,))
        encoding_dim = int(input_dim / 2) - 1
        hidden_dim = int(encoding_dim / 2)

#         encoder = Dense(encoding_dim, activation="tanh", activity_regularizer=regularizers.l1(learning_rate))(input_layer)
        encoder = Dense(encoding_dim, activation="tanh")(inps)# 277
        encoder = Dense(hidden_dim, activation="relu")(encoder) #128
        decoder = Dense(hidden_dim, activation='tanh')(encoder) # 128
        decoder = Dense(input_dim, activation='relu')(decoder) #277
        model = Model(inputs=inps, outputs=decoder)
        model.compile(
            metrics = ['accuracy'],
            optimizer=RAdam(),
            loss=[loss_fn]
        )
        model.summary()
        
        return model
    else:
        raise Exception('mode can be ann or autoencoder')

In [105]:
# from imblearn.keras import balanced_batch_generator
# from imblearn.under_sampling import NearMiss

In [106]:
# training_generator, steps_per_epoch = balanced_batch_generator(
#             X=([X_tr.loc[:, catcols].values[:,k] for k in range(X_tr.loc[:,catcols].values.shape[1])] +
#             [X_tr.loc[:, numcols].values]), y=y_tr,sampler=NearMiss(), batch_size=2048)
# model_bce.fit_generator(generator = training_generator, steps_per_epoch=steps_per_epoch, epochs=130, verbose=True, 
#             callbacks=[roc_callback(validation_data=([X_val.loc[:, catcols].values[:,k] for k in range(X_val.loc[:,catcols].values.shape[1])] +\
#                                                     [X_val.loc[:, numcols].values], y_val),mode='bce')]
#         )
#         model_focal.fit_generator(generator = training_generator, steps_per_epoch=steps_per_epoch, epochs=130, verbose=True, 
#             callbacks=[roc_callback(validation_data=([X_val.loc[:, catcols].values[:,k] for k in range(X_val.loc[:,catcols].values.shape[1])] +\
#                                                     [X_val.loc[:, numcols].values], y_val),mode='focal')]
#         )

#         print('balanced_pick steps_per_epoch',steps_per_epoch)

In [107]:
# autoencoder test
from time import time
from sklearn.model_selection import KFold, StratifiedKFold, TimeSeriesSplit, GroupKFold
import datetime
NFOLD = 5
folds =  GroupKFold(n_splits=NFOLD)
split_groups = train_group
training_start_time = time()
EPOCH_MAX = 400
for fold, (trn_idx, test_idx) in enumerate(folds.split(X, y, groups = split_groups)):
        
    print('fold {}'.format(fold))
    X_tr = X.iloc[trn_idx]
    y_tr = y.iloc[trn_idx]
    trn_idx_1 = y_tr.loc[y_tr==1].index
    trn_idx_0 = y_tr.loc[y_tr==0].index
    X_val = X.iloc[test_idx]
    y_val = y.iloc[test_idx]

    model_bce = create_model('binary_crossentropy')
    model_focal = create_model('focal_loss_fn')
    
    
    roc_best = 0.0
    for epoch in range(EPOCH_MAX):
        
        trn_idx_rnd = np.concatenate([np.random.choice(trn_idx_0,size=np.sum(y.iloc[trn_idx]),replace=False) ,trn_idx_1])
#         print(len(trn_idx_0), len(trn_idx_1), len(trn_idx_rnd))
        X_tr = X.iloc[trn_idx_rnd]
        y_tr = y.iloc[trn_idx_rnd]
#         print('train_all', len(y_tr),np.sum(y_tr),'val_all',len(y_val),np.sum(y_val))


        model_bce.fit(
            x=[X_tr.loc[:, catcols].values[:,k] for k in range(X_tr.loc[:,catcols].values.shape[1])] +\
                                                    [X_tr.loc[:, numcols].values]
                                                    , y=y_tr, epochs=1,batch_size=2048,verbose=False
        )
        if (epoch % 10)==0 :
            
            y_pred_val = model_bce.predict([X_val.loc[:, catcols].values[:,k] for k in range(X_val.loc[:,catcols].values.shape[1])] +\
                                                        [X_val.loc[:, numcols].values])
            roc_val = roc_auc_score(y_val, y_pred_val)
            if roc_best < roc_val:
                print('\r Epoch{} best model improved {} over {} \n'.format(epoch,roc_val,roc_best,))
                roc_best = roc_val
                model_bce.save("downsampling_catcoldiffall_remove_minusnumcols_account_Ds_focalg3a0.2_addr1_addr2_real31cat_{}_hlayer1{}_hlayer2{}_dr1{}_dr2{}_fold{}.h5".format('bce',hlayer1,hlayer2,dr1,dr2,fold))
            else:
                print('\r Epoch{}\n'.format(epoch,))
        else:
            print('\r Epoch{}\n'.format(epoch,))
            
    roc_best = 0.0
    for epoch in range(EPOCH_MAX):
        
        trn_idx_rnd = np.concatenate([np.random.choice(trn_idx_0,size=np.sum(y.iloc[trn_idx]),replace=False) ,trn_idx_1])
#         print(len(trn_idx_0), len(trn_idx_1), len(trn_idx_rnd))
        X_tr = X.iloc[trn_idx_rnd]
        y_tr = y.iloc[trn_idx_rnd]
#         print('train_all', len(y_tr),np.sum(y_tr),'val_all',len(y_val),np.sum(y_val))


        model_focal.fit(
            x=[X_tr.loc[:, catcols].values[:,k] for k in range(X_tr.loc[:,catcols].values.shape[1])] +\
                                                    [X_tr.loc[:, numcols].values]
                                                    , y=y_tr, epochs=1,batch_size=2048,verbose=False
        )
        if (epoch % 10)==0 :
            
            y_pred_val = model_focal.predict([X_val.loc[:, catcols].values[:,k] for k in range(X_val.loc[:,catcols].values.shape[1])] +\
                                                        [X_val.loc[:, numcols].values])
            roc_val = roc_auc_score(y_val, y_pred_val)
            if roc_best < roc_val:
                print('\r Epoch{} best model improved {} over {} \n'.format(epoch,roc_val,roc_best,))
                roc_best = roc_val
                model_focal.save("downsampling_catcoldiffall_remove_minusnumcols_account_Ds_focalg3a0.2_addr1_addr2_real31cat_{}_hlayer1{}_hlayer2{}_dr1{}_dr2{}_fold{}.h5".format('focal',hlayer1,hlayer2,dr1,dr2,fold))
            else:
                print('\r Epoch{}\n'.format(epoch,))
        else:
            print('\r Epoch{}\n'.format(epoch,))

        
    model_bce = load_model("downsampling_catcoldiffall_remove_minusnumcols_account_Ds_focalg3a0.2_addr1_addr2_real31cat_{}_hlayer1{}_hlayer2{}_dr1{}_dr2{}_fold{}.h5".format('bce',hlayer1,hlayer2,dr1,dr2,fold),custom_objects={'RAdam':RAdam,'binary_crossentropy_with_ranking':binary_crossentropy_with_ranking})
    model_focal= load_model("downsampling_catcoldiffall_remove_minusnumcols_account_Ds_focalg3a0.2_addr1_addr2_real31cat_{}_hlayer1{}_hlayer2{}_dr1{}_dr2{}_fold{}.h5".format('focal',hlayer1,hlayer2,dr1,dr2,fold),custom_objects={'RAdam':RAdam,'binary_crossentropy_with_ranking':binary_crossentropy_with_ranking})

    val_preds_bce = model_bce.predict([X_val.loc[:, catcols].values[:,k] for k in range(X_val.loc[:,catcols].values.shape[1])] +\
                                                [X_val.loc[:, numcols].values]).flatten()
    val_preds_focal = model_focal.predict([X_val.loc[:, catcols].values[:,k] for k in range(X_val.loc[:,catcols].values.shape[1])] +\
                                                [X_val.loc[:, numcols].values]).flatten()
    from scipy.stats import rankdata, spearmanr

    print('BCE preds: ', roc_auc_score(y_val, val_preds_bce))
    print('Focal preds: ',roc_auc_score(y_val, val_preds_focal))
    print('Correlation matrix: ')
    print(np.corrcoef(val_preds_bce, val_preds_focal))
    print("Spearman's correlation: ", spearmanr(val_preds_bce, val_preds_focal).correlation)
    print('Averaging: ', roc_auc_score(y_val, val_preds_bce + val_preds_focal))
    print('Rank averaging: ', roc_auc_score(y_val, rankdata(val_preds_bce, method='dense') + rankdata(val_preds_focal, method='dense')))

fold 0
after cat 23
after num 24
after cat 23
after num 24
 Epoch0 best model improved 0.8093581642203129 over 0.0 

 Epoch1

 Epoch2

 Epoch3

 Epoch4

 Epoch5

 Epoch6

 Epoch7

 Epoch8

 Epoch9

 Epoch10 best model improved 0.8627350282556662 over 0.8093581642203129 

 Epoch11

 Epoch12

 Epoch13

 Epoch14

 Epoch15

 Epoch16

 Epoch17

 Epoch18

 Epoch19

 Epoch20

 Epoch21

 Epoch22

 Epoch23

 Epoch24

 Epoch25

 Epoch26

 Epoch27

 Epoch28

 Epoch29

 Epoch30

 Epoch31

 Epoch32

 Epoch33

 Epoch34

 Epoch35

 Epoch36

 Epoch37

 Epoch38

 Epoch39

 Epoch40 best model improved 0.8864197802403752 over 0.8627350282556662 

 Epoch41

 Epoch42

 Epoch43

 Epoch44

 Epoch45

 Epoch46

 Epoch47

 Epoch48

 Epoch49

 Epoch50

 Epoch51

 Epoch52

 Epoch53

 Epoch54

 Epoch55

 Epoch56

 Epoch57

 Epoch58

 Epoch59

 Epoch60 best model improved 0.8899090310001084 over 0.8864197802403752 

 Epoch61

 Epoch62

 Epoch63

 Epoch64

 Epoch65

 Epoch66

 Epoch67

 Epoch68

 Epoch69

 Epoch70



 Epoch299

 Epoch300

 Epoch301

 Epoch302

 Epoch303

 Epoch304

 Epoch305

 Epoch306

 Epoch307

 Epoch308

 Epoch309

 Epoch310

 Epoch311

 Epoch312

 Epoch313

 Epoch314

 Epoch315

 Epoch316

 Epoch317

 Epoch318

 Epoch319

 Epoch320

 Epoch321

 Epoch322

 Epoch323

 Epoch324

 Epoch325

 Epoch326

 Epoch327

 Epoch328

 Epoch329

 Epoch330

 Epoch331

 Epoch332

 Epoch333

 Epoch334

 Epoch335

 Epoch336

 Epoch337

 Epoch338

 Epoch339

 Epoch340

 Epoch341

 Epoch342

 Epoch343

 Epoch344

 Epoch345

 Epoch346

 Epoch347

 Epoch348

 Epoch349

 Epoch350

 Epoch351

 Epoch352

 Epoch353

 Epoch354

 Epoch355

 Epoch356

 Epoch357

 Epoch358

 Epoch359

 Epoch360

 Epoch361

 Epoch362

 Epoch363

 Epoch364

 Epoch365

 Epoch366

 Epoch367

 Epoch368

 Epoch369

 Epoch370

 Epoch371

 Epoch372

 Epoch373

 Epoch374

 Epoch375

 Epoch376

 Epoch377

 Epoch378

 Epoch379

 Epoch380

 Epoch381

 Epoch382

 Epoch383

 Epoch384

 Epoch385

 Epoch386

 Epoch387

 Epoch388

 Epoch389


 Epoch75

 Epoch76

 Epoch77

 Epoch78

 Epoch79

 Epoch80 best model improved 0.9046094610270522 over 0.8980345596846336 

 Epoch81

 Epoch82

 Epoch83

 Epoch84

 Epoch85

 Epoch86

 Epoch87

 Epoch88

 Epoch89

 Epoch90

 Epoch91

 Epoch92

 Epoch93

 Epoch94

 Epoch95

 Epoch96

 Epoch97

 Epoch98

 Epoch99

 Epoch100 best model improved 0.9085244941057551 over 0.9046094610270522 

 Epoch101

 Epoch102

 Epoch103

 Epoch104

 Epoch105

 Epoch106

 Epoch107

 Epoch108

 Epoch109

 Epoch110 best model improved 0.9088998420666575 over 0.9085244941057551 

 Epoch111

 Epoch112

 Epoch113

 Epoch114

 Epoch115

 Epoch116

 Epoch117

 Epoch118

 Epoch119

 Epoch120 best model improved 0.9109461928132969 over 0.9088998420666575 

 Epoch121

 Epoch122

 Epoch123

 Epoch124

 Epoch125

 Epoch126

 Epoch127

 Epoch128

 Epoch129

 Epoch130

 Epoch131

 Epoch132

 Epoch133

 Epoch134

 Epoch135

 Epoch136

 Epoch137

 Epoch138

 Epoch139

 Epoch140

 Epoch141

 Epoch142

 Epoch143

 Epoch144


 Epoch301

 Epoch302

 Epoch303

 Epoch304

 Epoch305

 Epoch306

 Epoch307

 Epoch308

 Epoch309

 Epoch310

 Epoch311

 Epoch312

 Epoch313

 Epoch314

 Epoch315

 Epoch316

 Epoch317

 Epoch318

 Epoch319

 Epoch320

 Epoch321

 Epoch322

 Epoch323

 Epoch324

 Epoch325

 Epoch326

 Epoch327

 Epoch328

 Epoch329

 Epoch330

 Epoch331

 Epoch332

 Epoch333

 Epoch334

 Epoch335

 Epoch336

 Epoch337

 Epoch338

 Epoch339

 Epoch340

 Epoch341

 Epoch342

 Epoch343

 Epoch344

 Epoch345

 Epoch346

 Epoch347

 Epoch348

 Epoch349

 Epoch350

 Epoch351

 Epoch352

 Epoch353

 Epoch354

 Epoch355

 Epoch356

 Epoch357

 Epoch358

 Epoch359

 Epoch360

 Epoch361

 Epoch362

 Epoch363

 Epoch364

 Epoch365

 Epoch366

 Epoch367

 Epoch368

 Epoch369

 Epoch370

 Epoch371

 Epoch372

 Epoch373

 Epoch374

 Epoch375

 Epoch376

 Epoch377

 Epoch378

 Epoch379

 Epoch380

 Epoch381

 Epoch382

 Epoch383

 Epoch384

 Epoch385

 Epoch386

 Epoch387

 Epoch388

 Epoch389

 Epoch390

 Epoch391


 Epoch114

 Epoch115

 Epoch116

 Epoch117

 Epoch118

 Epoch119

 Epoch120

 Epoch121

 Epoch122

 Epoch123

 Epoch124

 Epoch125

 Epoch126

 Epoch127

 Epoch128

 Epoch129

 Epoch130

 Epoch131

 Epoch132

 Epoch133

 Epoch134

 Epoch135

 Epoch136

 Epoch137

 Epoch138

 Epoch139

 Epoch140 best model improved 0.9056266439996566 over 0.9048777414496523 

 Epoch141

 Epoch142

 Epoch143

 Epoch144

 Epoch145

 Epoch146

 Epoch147

 Epoch148

 Epoch149

 Epoch150

 Epoch151

 Epoch152

 Epoch153

 Epoch154

 Epoch155

 Epoch156

 Epoch157

 Epoch158

 Epoch159

 Epoch160 best model improved 0.9067542525824153 over 0.9056266439996566 

 Epoch161

 Epoch162

 Epoch163

 Epoch164

 Epoch165

 Epoch166

 Epoch167

 Epoch168

 Epoch169

 Epoch170 best model improved 0.9072501530260082 over 0.9067542525824153 

 Epoch171

 Epoch172

 Epoch173

 Epoch174

 Epoch175

 Epoch176

 Epoch177

 Epoch178

 Epoch179

 Epoch180

 Epoch181

 Epoch182

 Epoch183

 Epoch184

 Epoch185

 Epoch186

 Epoc

 Epoch337

 Epoch338

 Epoch339

 Epoch340

 Epoch341

 Epoch342

 Epoch343

 Epoch344

 Epoch345

 Epoch346

 Epoch347

 Epoch348

 Epoch349

 Epoch350

 Epoch351

 Epoch352

 Epoch353

 Epoch354

 Epoch355

 Epoch356

 Epoch357

 Epoch358

 Epoch359

 Epoch360 best model improved 0.9112055542548234 over 0.911026783656283 

 Epoch361

 Epoch362

 Epoch363

 Epoch364

 Epoch365

 Epoch366

 Epoch367

 Epoch368

 Epoch369

 Epoch370 best model improved 0.9113586398613911 over 0.9112055542548234 

 Epoch371

 Epoch372

 Epoch373

 Epoch374

 Epoch375

 Epoch376

 Epoch377

 Epoch378

 Epoch379

 Epoch380 best model improved 0.9129552868299355 over 0.9113586398613911 

 Epoch381

 Epoch382

 Epoch383

 Epoch384

 Epoch385

 Epoch386

 Epoch387

 Epoch388

 Epoch389

 Epoch390

 Epoch391

 Epoch392

 Epoch393

 Epoch394

 Epoch395

 Epoch396

 Epoch397

 Epoch398

 Epoch399

BCE preds:  0.9167761878002282
Focal preds:  0.9129552868299355
Correlation matrix: 
[[1.         0.78496525]
 [0.78

 Epoch127

 Epoch128

 Epoch129

 Epoch130 best model improved 0.9029522799616158 over 0.9002880757921308 

 Epoch131

 Epoch132

 Epoch133

 Epoch134

 Epoch135

 Epoch136

 Epoch137

 Epoch138

 Epoch139

 Epoch140

 Epoch141

 Epoch142

 Epoch143

 Epoch144

 Epoch145

 Epoch146

 Epoch147

 Epoch148

 Epoch149

 Epoch150 best model improved 0.9044866505544702 over 0.9029522799616158 

 Epoch151

 Epoch152

 Epoch153

 Epoch154

 Epoch155

 Epoch156

 Epoch157

 Epoch158

 Epoch159

 Epoch160 best model improved 0.9092493760187308 over 0.9044866505544702 

 Epoch161

 Epoch162

 Epoch163

 Epoch164

 Epoch165

 Epoch166

 Epoch167

 Epoch168

 Epoch169

 Epoch170

 Epoch171

 Epoch172

 Epoch173

 Epoch174

 Epoch175

 Epoch176

 Epoch177

 Epoch178

 Epoch179

 Epoch180 best model improved 0.9095151650849129 over 0.9092493760187308 

 Epoch181

 Epoch182

 Epoch183

 Epoch184

 Epoch185

 Epoch186

 Epoch187

 Epoch188

 Epoch189

 Epoch190

 Epoch191

 Epoch192

 Epoch193

 Epoch1

In [ ]:
# autoencoder test
from time import time
from sklearn.model_selection import KFold, StratifiedKFold, TimeSeriesSplit, GroupKFold
import datetime
NFOLD = 5
folds =  GroupKFold(n_splits=NFOLD)
split_groups = train_group
training_start_time = time()
for fold, (trn_idx, test_idx) in enumerate(folds.split(X, y, groups = split_groups)):
        
    print('fold {}'.format(fold))
    y_tr = y.iloc[trn_idx]
    trn_idx_1 = y_tr.loc[y_tr==1].index
    trn_idx_0 = y_tr.loc[y_tr==0].index
    print(len(trn_idx_0), len(trn_idx_1), len(trn_idx_rnd))
    X_val = X.iloc[test_idx]
    y_val = y.iloc[test_idx]

    model_bce = create_model('binary_crossentropy')
    
    
    roc_best = 0.0
    for epoch in range(500):
        
        trn_idx_rnd = np.concatenate([np.random.choice(trn_idx_0,size=np.sum(y.iloc[trn_idx]),replace=False) ,trn_idx_1])
#         print(len(trn_idx_0), len(trn_idx_1), len(trn_idx_rnd))
        X_tr = X.iloc[trn_idx_rnd]
        y_tr = y.iloc[trn_idx_rnd]
#         print('train_all', len(y_tr),np.sum(y_tr),'val_all',len(y_val),np.sum(y_val))


        model_bce.fit(
            x=[X_tr.loc[:, catcols].values[:,k] for k in range(X_tr.loc[:,catcols].values.shape[1])] +\
                                                    [X_tr.loc[:, numcols].values]
                                                    , y=y_tr, epochs=1,batch_size=2048,verbose=False
        )
        if (epoch % 10)==0 :
            
            y_pred_val = model_bce.predict([X_val.loc[:, catcols].values[:,k] for k in range(X_val.loc[:,catcols].values.shape[1])] +\
                                                        [X_val.loc[:, numcols].values])
            roc_val = roc_auc_score(y_val, y_pred_val)
            if roc_best < roc_val:
                print('\r Epoch{} best model improved {} over {} \n'.format(epoch,roc_val,roc_best,))
                roc_best = roc_val
                model_bce.save("downsampling_catcoldiffall_focalg3a0.2_addr1_addr2_real31cat_{}_hlayer1{}_hlayer2{}_dr1{}_dr2{}_fold{}.h5".format('bce',hlayer1,hlayer2,dr1,dr2,fold))
            else:
                print('\r Epoch{}\n'.format(epoch,))
        else:
            print('\r Epoch{}\n'.format(epoch,))
            
    model_focal = create_model('focal_loss_fn')
    roc_best = 0.0
    for epoch in range(500):
        
        trn_idx_rnd = np.concatenate([np.random.choice(trn_idx_0,size=np.sum(y.iloc[trn_idx]),replace=False) ,trn_idx_1])
#         print(len(trn_idx_0), len(trn_idx_1), len(trn_idx_rnd))
        X_tr = X.iloc[trn_idx_rnd]
        y_tr = y.iloc[trn_idx_rnd]
#         print('train_all', len(y_tr),np.sum(y_tr),'val_all',len(y_val),np.sum(y_val))


        model_focal.fit(
            x=[X_tr.loc[:, catcols].values[:,k] for k in range(X_tr.loc[:,catcols].values.shape[1])] +\
                                                    [X_tr.loc[:, numcols].values]
                                                    , y=y_tr, epochs=1,batch_size=2048,verbose=False
        )
        if (epoch % 10)==0 :
            
            y_pred_val = model_focal.predict([X_val.loc[:, catcols].values[:,k] for k in range(X_val.loc[:,catcols].values.shape[1])] +\
                                                        [X_val.loc[:, numcols].values])
            roc_val = roc_auc_score(y_val, y_pred_val)
            if roc_best < roc_val:
                print('\r Epoch{} best model improved {} over {} \n'.format(epoch,roc_val,roc_best,))
                roc_best = roc_val
                model_focal.save("downsampling_catcoldiffall_focalg3a0.2_addr1_addr2_real31cat_{}_hlayer1{}_hlayer2{}_dr1{}_dr2{}_fold{}.h5".format('focal',hlayer1,hlayer2,dr1,dr2,fold))
            else:
                print('\r Epoch{}\n'.format(epoch,))
        else:
            print('\r Epoch{}\n'.format(epoch,))

        
    model_bce = load_model("downsampling_catcoldiffall_focalg3a0.2_addr1_addr2_real31cat_{}_hlayer1{}_hlayer2{}_dr1{}_dr2{}_fold{}.h5".format('bce',hlayer1,hlayer2,dr1,dr2,fold),custom_objects={'RAdam':RAdam,'binary_crossentropy_with_ranking':binary_crossentropy_with_ranking})
    model_focal= load_model("downsampling_catcoldiffall_focalg3a0.2_addr1_addr2_real31cat_{}_hlayer1{}_hlayer2{}_dr1{}_dr2{}_fold{}.h5".format('focal',hlayer1,hlayer2,dr1,dr2,fold),custom_objects={'RAdam':RAdam,'binary_crossentropy_with_ranking':binary_crossentropy_with_ranking})

    val_preds_bce = model_bce.predict([X_val.loc[:, catcols].values[:,k] for k in range(X_val.loc[:,catcols].values.shape[1])] +\
                                                [X_val.loc[:, numcols].values]).flatten()
    val_preds_focal = model_focal.predict([X_val.loc[:, catcols].values[:,k] for k in range(X_val.loc[:,catcols].values.shape[1])] +\
                                                [X_val.loc[:, numcols].values]).flatten()
    from scipy.stats import rankdata, spearmanr

    print('BCE preds: ', roc_auc_score(y_val, val_preds_bce))
    print('Focal preds: ',roc_auc_score(y_val, val_preds_focal))
    print('Correlation matrix: ')
    print(np.corrcoef(val_preds_bce, val_preds_focal))
    print("Spearman's correlation: ", spearmanr(val_preds_bce, val_preds_focal).correlation)
    print('Averaging: ', roc_auc_score(y_val, val_preds_bce + val_preds_focal))
    print('Rank averaging: ', roc_auc_score(y_val, rankdata(val_preds_bce, method='dense') + rankdata(val_preds_focal, method='dense')))

fold 0
436106 17113 34226
after cat 22
after num 23
 Epoch0 best model improved 0.7430736740452283 over 0.0 

 Epoch1

 Epoch2

 Epoch3

 Epoch4

 Epoch5

 Epoch6

 Epoch7

 Epoch8

 Epoch9

 Epoch10 best model improved 0.8786169479668902 over 0.7430736740452283 

 Epoch11

 Epoch12

 Epoch13

 Epoch14

 Epoch15

 Epoch16

 Epoch17

 Epoch18

 Epoch19

 Epoch20

 Epoch21

 Epoch22

 Epoch23

 Epoch24

 Epoch25

 Epoch26

 Epoch27

 Epoch28

 Epoch29

 Epoch30 best model improved 0.8936598808074467 over 0.8786169479668902 

 Epoch31

 Epoch32

 Epoch33

 Epoch34

 Epoch35

 Epoch36

 Epoch37

 Epoch38

 Epoch39

 Epoch40

 Epoch41

 Epoch42

 Epoch43

 Epoch44

 Epoch45

 Epoch46

 Epoch47

 Epoch48

 Epoch49

 Epoch50

 Epoch51

 Epoch52

 Epoch53

 Epoch54

 Epoch55

 Epoch56

 Epoch57

 Epoch58

 Epoch59

 Epoch60

 Epoch61

 Epoch62

 Epoch63

 Epoch64

 Epoch65

 Epoch66

 Epoch67

 Epoch68

 Epoch69

 Epoch70

 Epoch71

 Epoch72

 Epoch73

 Epoch74

 Epoch75

 Epoch76

 Epoch77

 

In [114]:
fixed_focalg3a0.2_addr1_addr2_real31cat_bce_hlayer1768_hlayer2256_dr10.3_dr20.2_fold0.h5
fixed_focalg3a0.2_real31cat_focal_hlayer1768_hlayer2256_dr10.3_dr20.2_fold0.h5

SyntaxError: invalid token (<ipython-input-114-7e3c86317b21>, line 1)

In [89]:
fold = 0
model_bce = load_model("fixed_focalg3a0.2_addr1_addr2_real31cat_{}_hlayer1{}_hlayer2{}_dr1{}_dr2{}_fold{}.h5".format('bce',hlayer1,hlayer2,dr1,dr2,fold),custom_objects={'RAdam':RAdam,'binary_crossentropy_with_ranking':binary_crossentropy_with_ranking})
model_focal= load_model("fixed_focalg3a0.2_addr1_addr2_real31cat_{}_hlayer1{}_hlayer2{}_dr1{}_dr2{}_fold{}.h5".format('focal',hlayer1,hlayer2,dr1,dr2,fold),custom_objects={'RAdam':RAdam,'binary_crossentropy_with_ranking':binary_crossentropy_with_ranking})

val_preds_bce = model_bce.predict([X_val.loc[:, catcols].values[:,k] for k in range(X_val.loc[:,catcols].values.shape[1])] +\
                                            [X_val.loc[:, numcols].values]).flatten()
val_preds_focal = model_focal.predict([X_val.loc[:, catcols].values[:,k] for k in range(X_val.loc[:,catcols].values.shape[1])] +\
                                            [X_val.loc[:, numcols].values]).flatten()
from scipy.stats import rankdata, spearmanr

print('BCE preds: ', roc_auc_score(y_val, val_preds_bce))
print('Focal preds: ',roc_auc_score(y_val, val_preds_focal))
print('Correlation matrix: ')
print(np.corrcoef(val_preds_bce, val_preds_focal))
print("Spearman's correlation: ", spearmanr(val_preds_bce, val_preds_focal).correlation)
print('Averaging: ', roc_auc_score(y_val, val_preds_bce + val_preds_focal))
print('Rank averaging: ', roc_auc_score(y_val, rankdata(val_preds_bce, method='dense') + rankdata(val_preds_focal, method='dense')))

OSError: Unable to open file (unable to open file: name = 'fixed_focalg3a0.2_addr1_addr2_real31cat_bce_hlayer1768_hlayer2256_dr10.3_dr20.2_fold0.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
bce_preds_folds = []
focal_preds_folds = []
for fold in tqdm(range(NFOLD)):
    model_bce = load_model("fixed_focalg5a0.2_real31cat_{}_hlayer1{}_hlayer2{}_dr1{}_dr2{}_fold{}.h5".format('bce',hlayer1,hlayer2,dr1,dr2,fold),custom_objects={'RAdam':RAdam,'binary_crossentropy_with_ranking':binary_crossentropy_with_ranking})
    model_focal= load_model("fixed_focalg5a0.2_real31cat_{}_hlayer1{}_hlayer2{}_dr1{}_dr2{}_fold{}.h5".format('focal',hlayer1,hlayer2,dr1,dr2,fold),custom_objects={'RAdam':RAdam,'binary_crossentropy_with_ranking':binary_crossentropy_with_ranking})
    bce_preds = model_bce.predict([X_test.loc[:, catcols].values[:,k] for k in range(X_test.loc[:,catcols].values.shape[1])] +\
                                                    [X_test.loc[:, numcols].values]).flatten()
    focal_preds = model_focal.predict([X_test.loc[:, catcols].values[:,k] for k in range(X_test.loc[:,catcols].values.shape[1])] +\
                                                    [X_test.loc[:, numcols].values]).flatten()
    bce_preds_folds.append(bce_preds)
    focal_preds_folds.append(focal_preds)


In [ ]:
len(bce_preds_folds)

In [ ]:
bce_preds_folds[0]

In [ ]:
sub.isFraud = bce_preds_folds[0]+bce_preds_folds[1]+bce_preds_folds[2]+bce_preds_folds[3]+bce_preds_folds[4]+\
                focal_preds_folds[0]+focal_preds_folds[1]+focal_preds_folds[2]+focal_preds_folds[3]+focal_preds_folds[4]


In [ ]:
sub.isFraud.max()

In [ ]:
(bce_preds_folds[0]+focal_preds_folds[0]).max()

In [ ]:
sub.isFraud = sub.isFraud/(sub.isFraud.max())
sub.to_csv('submission_group_5folds_test_flossg3a0_2_{}_hlayer1{}_hlayer2{}_dr1{}_dr2{}.csv'.format('average',hlayer1,hlayer2,dr1,dr2), index=False)


In [ ]:
sub.iloc[range(199155,199175),]

In [ ]:
sub.iloc[231]

In [ ]:
sub.isFraud = rankdata(bce_preds_folds[0], method='dense')+rankdata(bce_preds_folds[1], method='dense')+rankdata(bce_preds_folds[2], method='dense')+rankdata(bce_preds_folds[3], method='dense')+rankdata(bce_preds_folds[4], method='dense')+\
                rankdata(focal_preds_folds[0], method='dense')+rankdata(focal_preds_folds[1], method='dense')+rankdata(focal_preds_folds[2], method='dense')+rankdata(focal_preds_folds[3], method='dense')+rankdata(focal_preds_folds[4], method='dense')

In [ ]:
 (rankdata(bce_preds_folds[0], method='dense')+rankdata(focal_preds_folds[0], method='dense')).max()

In [ ]:
sub.isFraud.max()

In [ ]:
sub.isFraud = sub.isFraud/sub.isFraud.max()
sub.to_csv('submission_group_5folds_test_flossg3a0_2_{}_hlayer1{}_hlayer2{}_dr1{}_dr2{}.csv'.format('rank',hlayer1,hlayer2,dr1,dr2), index=False)

In [ ]:
sub.iloc[231]

In [ ]:
len(sub)